# T07: Transformación de Datos a RDF (Grafo de Conocimiento)

Este notebook realiza el proceso de **ETL (Extract, Transform, Load)** para convertir los datos abiertos de los distritos de Valencia en un **Grafo de Conocimiento** enriquecido.

### Objetivos:
1.  **Cargar** los datos de distritos desde un archivo CSV.
2.  **Transformar** los datos al modelo RDF utilizando el vocabulario **schema.org**.
3.  **Enriquecer** los datos enlazándolos con **Wikidata** (Linked Data).
4.  **Generar** un archivo `.ttl` (Turtle) listo para ser consultado.

In [10]:
# 1. Importación de Librerías
from rdflib import Graph, URIRef, Literal, Namespace
from rdflib.namespace import RDF, DCTERMS, DC, SKOS, XSD, OWL
import pandas as pd
import os   
import unicodedata
import re
import json

# Definición del Dominio Base para nuestros recursos
domain = 'https://valencia.example.org/' 
print("Librerías importadas y dominio configurado.")

Librerías importadas y dominio configurado.


## 2. Funciones Auxiliares y Mapeo
Definimos funciones para normalizar texto y crear URIs válidas. Además, establecemos un **diccionario de mapeo** manual para conectar nuestros distritos con sus identificadores únicos en **Wikidata** (QIDs).

In [11]:
# --- FUNCIONES AUXILIARES ---

# Función para crear slugs seguros (URIs limpias)
def slugify(text):
    text = unicodedata.normalize("NFD", text)
    text = text.encode("ascii", "ignore").decode("utf-8")
    text = re.sub(r"[^a-zA-Z0-9\-]", "-", text)
    text = re.sub(r"-+", "-", text)
    return text.lower().strip("-")

# Función de normalización para coincidencia robusta
def normalize_for_match(text):
    if not isinstance(text, str): return ""
    text = unicodedata.normalize("NFD", text)
    text = text.encode("ascii", "ignore").decode("utf-8")
    return text.upper().strip()

# --- MAPEO EXPLÍCITO: Diccionario manual de Wikidata ---
WIKIDATA_MAPPING = {
    "CIUTAT VELLA": "Q3392733", 
    "L'EIXAMPLE": "Q3392791",   
    "EXTRAMURS": "Q3393056",      
    "CAMPANAR": "Q3393032",     
    "LA SAIDIA": "Q3393007",   
    "EL PLA DEL REAL": "Q4452090",
    "L'OLIVERETA": "Q3392792",  
    "PATRAIX": "Q55226",      
    "JESUS": "Q3393002",        
    "QUATRE CARRERES": "Q3393088",
    "POBLATS MARITIMS": "Q3392780",
    "CAMINS AL GRAU": "Q3392782",
    "ALGIROS": "Q3392835",     
    "BENIMACLET": "Q536707",   
    "RASCANYA": "Q3392759",     
    "BENICALAP": "Q777986",   
    "POBLATS DEL NORD": "Q3393120",
    "POBLATS DE L'OEST": "Q3393115",
    "POBLATS DEL SUD": "Q3392798",  
}
# Normalizamos las claves
WIKIDATA_MAPPING = {normalize_for_match(k): v for k, v in WIKIDATA_MAPPING.items()}
print("Funciones y mapeo Wikidata listos.")

Funciones y mapeo Wikidata listos.


## 3. Inicialización del Grafo RDF
Creamos un grafo vacío y definimos los **Namespaces** (vocabularios) que vamos a utilizar.
*   **schema**: Vocabulario principal para describir lugares.
*   **owl/skos**: Para relaciones semánticas y jerarquías.
*   **geo**: Para datos geoespaciales.

In [12]:
# Instancia el grafo y los namespaces
g = Graph()
g.bind("rdf", RDF)
g.bind("dcterms", DCTERMS)
g.bind("dc", DC)
g.bind("skos", SKOS)
g.bind("xsd", XSD)
g.bind("owl", OWL)

# Namespace principal schema.org
schema = Namespace("https://schema.org/")
g.bind("schema", schema)

# Namespace para GeoSPARQL
GEO = Namespace("http://www.opengis.net/ont/geosparql#")
g.bind("geo", GEO)

print("Grafo inicializado con Namespaces.")

Grafo inicializado con Namespaces.


## 4. Carga de Datos (Extract)
Leemos el archivo CSV original que contiene la información de los distritos.

In [13]:
print("--- Cargando y procesando: districtes-distritos.csv ---")

# Ajuste de rutas
csv_path = '../data/districtes-distritos.csv'
if not os.path.exists(csv_path):
    csv_path = 'data/districtes-distritos.csv'

try:
    df_distritos = pd.read_csv (csv_path, sep=';')
    print(f"CSV cargado con éxito desde {csv_path}. Total de filas: {len(df_distritos)}")
except FileNotFoundError:
    print(f"Error: No se encuentra el archivo en {csv_path}")

--- Cargando y procesando: districtes-distritos.csv ---
CSV cargado con éxito desde ../data/districtes-distritos.csv. Total de filas: 19


## 5. Transformación a RDF (Transform)
Iteramos sobre cada fila del CSV para crear recursos RDF.
Para cada distrito:
1.  Creamos una **URI única** (ej: `.../district/ciutat-vella`).
2.  Asignamos el tipo `schema:Place`.
3.  Añadimos propiedades: nombre, identificador.
4.  **Enriquecemos** con el enlace a Wikidata (`sameAs`).
5.  Añadimos información geográfica (polígono y coordenadas).

In [14]:
count_processed = 0

print("Iniciando transformación de datos...")

for index, row in df_distritos.iterrows():
    # Sanitiza el nombre y código
    nombre_distrito = row["Nombre"].strip()
    nombre_key = normalize_for_match(nombre_distrito)
    
    # Filtrar filas inválidas
    if nombre_key not in WIKIDATA_MAPPING:
        continue
    
    count_processed += 1
    codigo_distrito = str(row["Código distrito"]).strip()
    
    # URI del recurso
    place_uri_name = slugify(nombre_distrito)
    place_uri = URIRef(domain + 'district/' + place_uri_name)
    
    # 1. Definición del Tipo (schema:Place)
    g.add((place_uri, RDF.type, schema.Place))
    g.add((place_uri, RDF.type, SKOS.Concept))
    
    # 2. Propiedades Básicas
    g.add((place_uri, schema.name, Literal(nombre_distrito, lang="ca")))
    g.add((place_uri, SKOS.prefLabel, Literal(nombre_distrito, lang="ca")))
    g.add((place_uri, schema.identifier, Literal(codigo_distrito)))

    # 3. Enlace con Wikidata (sameAs)
    qid = WIKIDATA_MAPPING.get(nombre_key)
    if qid:
        wikidata_uri = URIRef(f"http://www.wikidata.org/entity/{qid}")
        g.add((place_uri, schema.sameAs, wikidata_uri)) 
        g.add((place_uri, OWL.sameAs, wikidata_uri))

    # 4. Geometría: Polígono (GeoShape)
    geo_shape_json = None
    
    # Intentamos usar el del CSV primero (incluso si es inválido, para luego corregir manualmente)
    if pd.notnull(row["geo_shape"]):
        geo_shape_json = row["geo_shape"]
        # Verificación opcional solo para log
        try:
            json.loads(row["geo_shape"])
        except json.JSONDecodeError:
            print(f"⚠️ Polígono corrupto en CSV para '{nombre_distrito}'. Se usará tal cual y se corregirá manualmente si es necesario.")

    # Si finalmente tenemos geometría, la añadimos
    if geo_shape_json:
        shape_uri = URIRef(place_uri + "/geoshape")
        g.add((place_uri, schema.geo, shape_uri))
        g.add((shape_uri, RDF.type, schema.GeoShape))
        g.add((shape_uri, schema.polygon, Literal(geo_shape_json))) 

    # 5. Geometría: Punto Central (GeoCoordinates)
    if pd.notnull(row["geo_point_2d"]):
        lat_lon = row["geo_point_2d"].split(',')
        lat = lat_lon[0].strip()
        lon = lat_lon[1].strip()
        
        geo_coord = URIRef(place_uri + "/center")
        g.add((place_uri, schema.geo, geo_coord)) 
        g.add((geo_coord, RDF.type, schema.GeoCoordinates))
        g.add((geo_coord, schema.latitude, Literal(lat, datatype=XSD.decimal)))
        g.add((geo_coord, schema.longitude, Literal(lon, datatype=XSD.decimal)))

print(f"Transformación completada. Procesados {count_processed} distritos.")

Iniciando transformación de datos...
⚠️ Polígono corrupto en CSV para 'POBLATS MARITIMS'. Se usará tal cual y se corregirá manualmente si es necesario.
⚠️ Polígono corrupto en CSV para 'POBLATS DEL SUD'. Se usará tal cual y se corregirá manualmente si es necesario.
Transformación completada. Procesados 19 distritos.


## 6. Serialización y Verificación (Load)
Guardamos el grafo resultante en un archivo `.ttl` (Turtle) y realizamos una comprobación automática para asegurar que se han procesado todos los registros correctamente.

In [15]:
# Guardamos el resultado en formato Turtle (ttl)
output_dir = "../rdf"
if not os.path.exists("../rdf"):
    output_dir = "rdf"

output_file_name = "valencia_districts_places_enriched.ttl"
output_file = os.path.join(output_dir, output_file_name)

os.makedirs(output_dir, exist_ok=True) 

# print(f"Grafo RDF guardado en: {output_file}")
print("⏳ Serialización pospuesta hasta después de las correcciones manuales...")
print(f"Total de triples (antes de corrección): {len(g)}")

# --- VERIFICACIÓN ---
num_places = len(list(g.subjects(RDF.type, schema.Place)))
if num_places == count_processed:
    print(f"✅ ÉXITO: Se generaron {num_places} recursos schema:Place (coincide con filas procesadas).")
else:
    print(f"⚠️ ALERTA: Generados {num_places} recursos vs {count_processed} filas procesadas.")

# Ejemplo de visualización
print("\nEjemplo de Triples (Ciutat Vella):")
example_uri = URIRef(domain + 'district/ciutat-vella')
for s, p, o in g.triples((example_uri, None, None)):
    print(f"  {p.split('/')[-1]} -> {o}")

⏳ Serialización pospuesta hasta después de las correcciones manuales...
Total de triples (antes de corrección): 266
✅ ÉXITO: Se generaron 19 recursos schema:Place (coincide con filas procesadas).

Ejemplo de Triples (Ciutat Vella):
  22-rdf-syntax-ns#type -> https://schema.org/Place
  22-rdf-syntax-ns#type -> http://www.w3.org/2004/02/skos/core#Concept
  name -> CIUTAT VELLA
  core#prefLabel -> CIUTAT VELLA
  identifier -> 1
  sameAs -> http://www.wikidata.org/entity/Q3392733
  owl#sameAs -> http://www.wikidata.org/entity/Q3392733
  geo -> https://valencia.example.org/district/ciutat-vella/geoshape
  geo -> https://valencia.example.org/district/ciutat-vella/center


### 🔧 Corrección Manual de Geometrías (Vía Código)

Debido a que los polígonos de **POBLATS MARITIMS** y **POBLATS DEL SUD** estaban corruptos en el CSV original y la recuperación automática falló, se han incorporado sus geometrías correctas **directamente en el código** en la siguiente celda.

De esta forma, al regenerar el archivo `.ttl`, las geometrías correctas se aplicarán automáticamente sin necesidad de editar el archivo final a mano.

In [16]:
# --- CORRECCIÓN MANUAL DE GEOMETRÍAS ---
# Definimos las geometrías correctas (GeoJSON) para los distritos con datos corruptos.

# 1. POBLATS MARITIMS
POLY_MARITIMS = """
{"coordinates": [[[-0.3315104329274244, 39.48560696385567], [-0.33149725881557573, 39.48560731894289], [-0.3313862689627255, 39.485594482001666], [-0.33129296144256687, 39.48558166369558], [-0.3311253877287801, 39.485540275553774], [-0.33105893987893553, 39.48551094842113], [-0.33100590969710886, 39.48548492935352], [-0.33084897880218267, 39.485424272401545], [-0.33076214844450386, 39.48536492830721], [-0.3307009152109501, 39.48531235215494], [-0.3306209628625099, 39.48522639206001], [-0.33052325879664946, 39.48513160093971], [-0.3304080053753943, 39.485022388987424], [-0.33030178994546344, 39.48496106770415], [-0.3302306509116984, 39.48492857836701], [-0.33012664306162964, 39.48490375690671], [-0.3300051093722095, 39.48486450607423], [-0.3298799514256867, 39.484825180841554], [-0.32978192362254044, 39.48480116313676], [-0.32964695947326433, 39.48476654908195], [-0.3295286103662824, 39.484731587018835], [-0.3293389359816405, 39.48466902240932], [-0.32921018915909517, 39.48462780322284], [-0.32916956640384754, 39.48461480683273], [-0.3288442995254434, 39.48450918006237], [-0.3283886887908473, 39.4843609803104], [-0.32778320038690356, 39.48419096334191], [-0.3277325370797275, 39.48418637490248], [-0.3275542019745818, 39.484170219810615], [-0.3272721999552128, 39.48416440928878], [-0.3271109071056324, 39.484165581893016], [-0.32671019482109526, 39.484185885136455], [-0.3265902222428608, 39.48418577348396], [-0.32646807645328485, 39.48416986447939], [-0.3264608461310439, 39.48416893237919], [-0.3264506317915936, 39.48416759829777], [-0.32640566257500886, 39.48415276225955], [-0.32632454390650767, 39.484136851181596], [-0.3262126194070483, 39.4841354468957], [-0.32564053924170044, 39.48411889418956], [-0.32542974335023667, 39.48412828514606], [-0.3253580608107125, 39.484115212125175], [-0.3252253671574286, 39.48408696933634], [-0.3250601548886882, 39.484006312856785], [-0.3248694721460619, 39.48379555155833], [-0.3246263418181736, 39.48354431343172], [-0.32450516898321813, 39.48339938403978], [-0.3242725058095423, 39.47997877959081], [-0.3243186544106152, 39.47826370037172], [-0.32427936840039684, 39.475650286167024], [-0.3245740653379311, 39.474558016875015], [-0.323813841008601, 39.47265320272458], [-0.3230630581262739, 39.468059515150365], [-0.32272781417912777, 39.46549333198227], [-0.3226069495755487, 39.46307622611391], [-0.32240399646154094, 39.4631030897403], [-0.32112603450930516, 39.463091664447724], [-0.3200810931892867, 39.463027042136176], [-0.31173900324781684, 39.45113670690517], [-0.3117019642043454, 39.45111783271192], [-0.31167235449066305, 39.451094625990365], [-0.31164038896147134, 39.45107586913307], [-0.3115989963120133, 39.45105689420298], [-0.31156338157206354, 39.45102003545409], [-0.31155008177724003, 39.45098819725117], [-0.31152246998995914, 39.450969541072794], [-0.3115074111895904, 39.45096468836309], [-0.3114750993512816, 39.45095493217934], [-0.31145564831635586, 39.45093196040042], [-0.3110818032999556, 39.450927825078026], [-0.3089219941639936, 39.45092740154098], [-0.30823126496422254, 39.45092942333387], [-0.3076332485783506, 39.450843510584725], [-0.30748537240246543, 39.450817563018205], [-0.3068419817742003, 39.45060897725551], [-0.305589520534218, 39.45020610387273], [-0.30393489705482096, 39.44966776658167], [-0.3025463471395917, 39.44921666009294], [-0.3024554043301894, 39.44927761036373], [-0.3024340452222815, 39.44930414076896], [-0.3023724254126919, 39.449338745409925], [-0.3022920894142913, 39.449350393792145], [-0.3022164298934861, 39.44933512451618], [-0.30211931419192695, 39.44929233097684], [-0.30204978250874426, 39.449250177685386], [-0.30200818374091687, 39.44921768150795], [-0.3019769286295191, 39.44918092107759], [-0.3019677771169549, 39.44913566504303], [-0.3019617379199896, 39.44910399433481], [-0.30218814023210294, 39.434128015836066], [-0.3021966003515768, 39.43398407241388], [-0.3022350681907815, 39.43389038251952], [-0.3023166779988115, 39.43378867615017], [-0.30243874930371906, 39.43369241311855], [-0.3025907920165527, 39.433628376070175], [-0.3027967510904961, 39.43359712052815], [-0.30302059086743793, 39.43362934066871], [-0.303125648551791, 39.43367230856762], [-0.30323908708681196, 39.433724488563556], [-0.30334292933878176, 39.433798967662], [-0.3033799071819533, 39.43389441729654], [-0.30338409847591635, 39.43401161912124], [-0.30334383392938224, 39.43422689469786], [-0.30328441037049253, 39.43426155099991], [-0.30310560689617494, 39.434266411245574], [-0.3029639914884963, 39.434267629742216], [-0.30287644372567835, 39.43429712888349], [-0.3028186810022215, 39.43434533658707], [-0.3026627101649442, 39.44281871741831], [-0.3026277840963601, 39.444700712430205], [-0.3034563394996801, 39.44471992951804], [-0.3037654081284557, 39.44472709974308], [-0.30378700068318454, 39.444732105010786], [-0.3038093129776129, 39.44473712697087], [-0.30382206208870066, 39.44476444889815], [-0.3038205860144897, 39.44485900627635], [-0.3038203993083542, 39.44495808891769], [-0.3038248896190487, 39.445048280337], [-0.30383383159810895, 39.44508001830859], [-0.3038398714059948, 39.445111688956224], [-0.3038373532515263, 39.44512063926651], [-0.30383102790496075, 39.44513400562165], [-0.30382534710956816, 39.44520594364258], [-0.3038239694654531, 39.44535455564889], [-0.3038230510227514, 39.44545363031778], [-0.3038218604443427, 39.44550315068585], [-0.3038260066426116, 39.445521264317975], [-0.30390823164886316, 39.44552317172237], [-0.3039423382540625, 39.44552396288904], [-0.3039922216748374, 39.445529624373], [-0.3040100188979476, 39.44553904592644], [-0.3040558963431542, 39.44553560573477], [-0.30405531349984233, 39.4455130704024], [-0.3042753230308141, 39.44551366010922], [-0.30427460404674905, 39.4456262525009], [-0.30423525069531837, 39.445629844099926], [-0.3040582277160078, 39.445625747063616], [-0.3040547426734215, 39.4456031444128], [-0.30400250308533067, 39.44560193266346], [-0.30398837613185214, 39.44561061369562], [-0.303957729379039, 39.445614407161386], [-0.3038329348025931, 39.44561151228322], [-0.30381287445299693, 39.446507405973115], [-0.3038062863715864, 39.446791027930914], [-0.3038070176269928, 39.44690365393464], [-0.3038350598492317, 39.44696736551974], [-0.3038910212624322, 39.44702270702241], [-0.3039244339239339, 39.44704149955092], [-0.3039803333119822, 39.44706081368866], [-0.30401553286414157, 39.44707063891792], [-0.3040864529755504, 39.447076788344894], [-0.30417787419154546, 39.44707890889065], [-0.30500769939516387, 39.4470846315766], [-0.30632688074528036, 39.447092682159976], [-0.30690549907243864, 39.447097073287026], [-0.30743677615907905, 39.44710487816311], [-0.3075588449362817, 39.44710320168561], [-0.30758199943116027, 39.44706770316268], [-0.30761037445957273, 39.447009803749204], [-0.30764098403170564, 39.44698799100546], [-0.3076565359281064, 39.446961325079734], [-0.3076976502374448, 39.44693074693298], [-0.30771758155779205, 39.446922199903405], [-0.3080644573709131, 39.44634466713941], [-0.308419373886162, 39.445708762685385], [-0.30992790433803236, 39.44298703281605], [-0.31034154712197093, 39.44223988078297], [-0.3103757218559242, 39.442182114819495], [-0.31040015064383836, 39.44215115854254], [-0.3104213312453789, 39.442129126825556], [-0.31334451408284136, 39.44187239179913], [-0.3133413721981313, 39.44184078857375], [-0.313390880339402, 39.44183742858059], [-0.3134014112271022, 39.44184668074186], [-0.3134093191142234, 39.44190542035907], [-0.3134017571015725, 39.44193227179478], [-0.31199547349979084, 39.44205741948482], [-0.31184405803277127, 39.4420674307422], [-0.311869110140035, 39.44228421991733], [-0.31193834551613503, 39.442560578735], [-0.3119901182674154, 39.44261132416142], [-0.3124186315865975, 39.442778877519025], [-0.3128213351316101, 39.44291880637578], [-0.3105190245389845, 39.447041085265795], [-0.31006665915835707, 39.447850402070024], [-0.30968897651844796, 39.44772005139492], [-0.3096692450478139, 39.44774211648431], [-0.3095599004219244, 39.447753098448814], [-0.3094920694731084, 39.44776053696294], [-0.30933012510269187, 39.447779309783215], [-0.30914808751664435, 39.44781113027002], [-0.30900704350805525, 39.447852908348565], [-0.30884498942100747, 39.447912226237165], [-0.30870470273159567, 39.447972038936626], [-0.3086104797593035, 39.44802390943644], [-0.30845903210921366, 39.448128507021096], [-0.30835380989756933, 39.448220661859985], [-0.30828538661906313, 39.448300155767], [-0.30822801449462306, 39.44837540121518], [-0.3081750073440296, 39.448450747728856], [-0.3081251281092319, 39.44855769720816], [-0.30810003970122657, 39.44862468158318], [-0.3080591683722743, 39.44879940511667], [-0.3080507171216709, 39.448943339991125], [-0.30804932182025613, 39.449073934210034], [-0.30804522367557563, 39.44921797891311], [-0.3080466552423202, 39.44931260369768], [-0.3080438087023682, 39.44944316429631], [-0.3080399019693171, 39.449695318135824], [-0.3080428862367724, 39.449844031232246], [-0.3080692633956453, 39.44998877277966], [-0.3081415467971902, 39.450148099672006], [-0.30825302458377213, 39.45030833428109], [-0.30836968570986467, 39.450428149563955], [-0.30849118321996766, 39.45051654619551], [-0.3086200961823334, 39.450582583601], [-0.30881377015279626, 39.450663642557664], [-0.3088783811360066, 39.45068315609447], [-0.30894572121178715, 39.45070723714918], [-0.30904444156278404, 39.45072754037211], [-0.3091522043515546, 39.450739044142225], [-0.30920065242143896, 39.4507446701964], [-0.30931693759398743, 39.45076087548912], [-0.30934271274356484, 39.450770480939156], [-0.3093674217624105, 39.450789061419414], [-0.30937069730926703, 39.450798145977316], [-0.30994135955634183, 39.450802347121474], [-0.30994112469328244, 39.4501672263903], [-0.30994023675421195, 39.450077127566715], [-0.31221429250844657, 39.450071156255326], [-0.3123495927610742, 39.45076345308218], [-0.3137868622568483, 39.450724593309275], [-0.31382756341993545, 39.45066697725253], [-0.3138698335775961, 39.450568858144024], [-0.3137459060404524, 39.4505434720358], [-0.3135566695667089, 39.44894907457846], [-0.3136928407252867, 39.44892069144376], [-0.3136958343183232, 39.44857842870281], [-0.31429592757602554, 39.44851572294668], [-0.3142771117739609, 39.44824953042786], [-0.3142801300329202, 39.44792528565265], [-0.3143595519185698, 39.4476163194749], [-0.31446546746694376, 39.44731697413443], [-0.3145428671764936, 39.447098057456216], [-0.3146044792808495, 39.44706344607291], [-0.3145584549641972, 39.44699481712965], [-0.3149347113099449, 39.44684134379231], [-0.32082254061247373, 39.4551570464253], [-0.32232926575155574, 39.45728626645927], [-0.32310441407025947, 39.45838065849786], [-0.3231273626388975, 39.45842623076796], [-0.3231864948764426, 39.4585131757701], [-0.32321759288339963, 39.4585544313594], [-0.32322779413761454, 39.45857268379448], [-0.32349061512576555, 39.45846161434923], [-0.3235608415615669, 39.45856232777359], [-0.3237864274751706, 39.458702653793125], [-0.32433920999581084, 39.458170353154514], [-0.324473383255122, 39.45804281542274], [-0.32454004448294094, 39.45799029762672], [-0.3246259067487911, 39.457929213069136], [-0.3247183477843631, 39.45788629736624], [-0.32482187095707077, 39.45785714979518], [-0.32490293383490326, 39.45784550262126], [-0.3249936395475084, 39.45784758144373], [-0.3250566079606611, 39.45785353515628], [-0.3251600730499142, 39.45788294275665], [-0.32526211728774335, 39.4579303350456], [-0.325353312568065, 39.45799549504776], [-0.3254278954781359, 39.45809630763319], [-0.3254643978433546, 39.45820525256141], [-0.3254635088992206, 39.45830432824727], [-0.3254386357124379, 39.458384834492755], [-0.3253966904703726, 39.4584559390986], [-0.325319566975206, 39.45853524282135], [-0.3248498797375937, 39.45898387791348], [-0.32411910212473216, 39.45968775305823], [-0.32431346285922247, 39.459809341643634], [-0.32430130598235063, 39.45989914010856], [-0.3240843253810827, 39.45998874498618], [-0.3244602060682442, 39.460528905627164], [-0.32507377575290936, 39.46026825427417], [-0.32519766993302374, 39.46044677625053], [-0.32458409906675306, 39.460707428213176], [-0.32501784498134606, 39.46132999782235], [-0.3251386050793324, 39.461476908131864], [-0.3251896810884338, 39.46152763176243], [-0.3255298155398069, 39.46097241379472], [-0.3255641331223863, 39.46098671664965], [-0.3252502250744609, 39.46150199902657], [-0.3252401068815855, 39.46151978362191], [-0.3255897690226279, 39.46164494438476], [-0.3257315340813725, 39.461413979540296], [-0.32591216522753175, 39.4614766927734], [-0.3259637282664247, 39.461495896696135], [-0.32584617908361013, 39.46168237536711], [-0.32587687782065666, 39.461791186596045], [-0.3258758862410849, 39.46183619845365], [-0.32596642452049107, 39.46184278609611], [-0.3259678454193518, 39.461824801316915], [-0.32600122905999207, 39.4618255694519], [-0.32598914625009784, 39.46195141374861], [-0.32642080983420957, 39.46198386706712], [-0.32644256153391854, 39.46179518399292], [-0.32860740757885143, 39.46195755813266], [-0.3286337779886957, 39.461723937047836], [-0.32867369904649474, 39.4617248546855], [-0.32869090156669717, 39.46171173699265], [-0.32872988764095845, 39.46169912000825], [-0.3287728721615649, 39.461695603656864], [-0.32895468071821826, 39.46170879117525], [-0.3290646047118689, 39.46170230884322], [-0.3291794110918253, 39.461682425500676], [-0.32930996834227305, 39.461649390897456], [-0.3294080226173977, 39.46161109585914], [-0.32949811082313546, 39.46157262669324], [-0.3295459344602184, 39.461537690654694], [-0.32957800720569147, 39.46151590578128], [-0.3296175408460005, 39.46150780544853], [-0.32963422683434285, 39.461508188857415], [-0.3297922844574687, 39.461610916736745], [-0.33021530732940435, 39.46124226832753], [-0.3310238611321102, 39.460535636265305], [-0.33126306797281063, 39.46032041529669], [-0.3313291350748789, 39.46024535807423], [-0.3311194941196722, 39.4601054127048], [-0.3311842475780252, 39.459931229184974], [-0.33142038819115915, 39.45981503364888], [-0.33060814033881747, 39.458832452986044], [-0.3308569647750093, 39.458707541136], [-0.3308672966063835, 39.4587032740515], [-0.33005324130468433, 39.45771164833366], [-0.33030479328211887, 39.457591304651544], [-0.3298897691312761, 39.45708628908296], [-0.32974049471697203, 39.45692521533692], [-0.3295521209017388, 39.45679926897421], [-0.32801291495857815, 39.45593060018819], [-0.32655384811999694, 39.457351957841745], [-0.32654095806244166, 39.457347156950206], [-0.3265101771092784, 39.45737347509478], [-0.32650184420704315, 39.45738229214636], [-0.32645692482539457, 39.45741729377526], [-0.3264360887729371, 39.457430327562975], [-0.3263907940194166, 39.45745631178738], [-0.32635163769114695, 39.45747342846875], [-0.32631847073835707, 39.457486178555826], [-0.32627587530940166, 39.457498720729504], [-0.32613187544295397, 39.45752243398507], [-0.32604061122981465, 39.457515829827024], [-0.3259072266133167, 39.45749023895935], [-0.3258106433702637, 39.457451981637526], [-0.32576668481035326, 39.45742394391116], [-0.3257136428435203, 39.45738668840903], [-0.3256961842339703, 39.45736826918474], [-0.3256592797378486, 39.45732688062754], [-0.32564054212248306, 39.45730392759247], [-0.32562489120036314, 39.4572765412137], [-0.3256079496474119, 39.45724462076221], [-0.3255909775767985, 39.457194682122456], [-0.3255858614589415, 39.45717654690669], [-0.32558071481289774, 39.45714039350453], [-0.32557956597967797, 39.457113340841964], [-0.325581503476643, 39.45708186383993], [-0.32558812846485424, 39.45704147695569], [-0.32559821531992234, 39.45700567410597], [-0.32561176403895653, 39.45697445528985], [-0.32562982754893516, 39.45693883599357], [-0.32565135291452146, 39.45690780072753], [-0.3256703623682212, 39.45688571630512], [-0.32569841300884256, 39.45685483118438], [-0.3257165186220176, 39.45683723032685], [-0.3258248074706732, 39.456740625927424], [-0.3260252470749964, 39.45655154987657], [-0.32693483944107793, 39.45567159084928], [-0.3270199459581239, 39.45559246957215], [-0.3268370296422476, 39.45547566214491], [-0.32692539694793954, 39.45531103284879], [-0.32284655771972215, 39.45402353390794], [-0.3229107181024046, 39.45390339381917], [-0.3229021529393865, 39.453880674668966], [-0.32290301538427085, 39.4538581726801], [-0.32290877906036936, 39.45384028796014], [-0.3229436397729023, 39.453764516649066], [-0.32304501543969216, 39.453564155015286], [-0.3230827784239084, 39.45348845050545], [-0.32309904596626793, 39.453461798966984], [-0.3231127974278087, 39.45344409821653], [-0.3231374393236438, 39.453426648299384], [-0.32317731388960214, 39.453409549219884], [-0.3231985241776449, 39.453405533366286], [-0.3232707134924384, 39.45339818726704], [-0.32328884874267333, 39.45339860494591], [-0.32379439028630025, 39.45339672497345], [-0.323811805706546, 39.453397125995124], [-0.32383150441065767, 39.453357040242835], [-0.3238186152795712, 39.45335223907774], [-0.32386241364084345, 39.453213612067046], [-0.3238883642828576, 39.4532187139829], [-0.323963713871957, 39.452977212862315], [-0.3239399344216927, 39.452972160957025], [-0.3239588713049439, 39.452914040148], [-0.32527447356222894, 39.45315151711919], [-0.32529779376023893, 39.45311151445187], [-0.32531012384258, 39.45311179821441], [-0.32534187969541406, 39.453117033405675], [-0.3253474819649788, 39.45310364921385], [-0.3263345290129825, 39.45340562255625], [-0.3264865788684446, 39.45324696290483], [-0.3264965231776909, 39.4532336785382], [-0.32545554704292007, 39.45289893494163], [-0.3254625889372887, 39.452885583872785], [-0.3254415090697759, 39.45286708127811], [-0.3254191924185338, 39.45286206333802], [-0.3255764123158242, 39.45256839264531], [-0.32555060349057396, 39.45254077250296], [-0.325561451614357, 39.452523004640604], [-0.3255900901698854, 39.45251465490659], [-0.3259654564348851, 39.45263139693368], [-0.3262614521100559, 39.452070655681425], [-0.3262257700877257, 39.45192119043723], [-0.32347392964235405, 39.45105159405814], [-0.3235968120497709, 39.4508201964775], [-0.32347122586078947, 39.45078127840899], [-0.3234766442281886, 39.45077239444332], [-0.3234379783216809, 39.45075799087133], [-0.3218124577339229, 39.45025210777914], [-0.3206760629139324, 39.4498880968534], [-0.3206440953591037, 39.44986934240676], [-0.3205807716593034, 39.44985436944189], [-0.3213873609560624, 39.44834598840361], [-0.32161549884004886, 39.44841880263129], [-0.3214824859699523, 39.44866798218786], [-0.32524794084940284, 39.44988075792224], [-0.3253590754144392, 39.449671609864225], [-0.3256463307659132, 39.449642176055136], [-0.3250641313642525, 39.44614693123756], [-0.3186107454623699, 39.444097454871425], [-0.318712998563893, 39.443892612612935], [-0.3186940029224261, 39.44344173697812], [-0.31777117300452995, 39.44343847077892], [-0.31773659463235676, 39.44356379526117], [-0.31765930665187586, 39.44383677839648], [-0.3177768727867852, 39.44385750905382], [-0.31774008305234835, 39.44396476500231], [-0.3175228988513071, 39.44392822209778], [-0.31755534726512347, 39.44382086602716], [-0.31762610459625773, 39.44383150777236], [-0.31773634729775635, 39.44343766709231], [-0.31626742010698966, 39.44343530719176], [-0.31626296229613593, 39.44347574361314], [-0.31609755192631617, 39.44347192412906], [-0.31609026495594167, 39.443359146548474], [-0.3188233429893562, 39.443363641950235], [-0.31883857342905497, 39.44336399328759], [-0.3188902983978454, 39.443284107727855], [-0.3189268102409592, 39.44322188871744], [-0.31895028528734126, 39.44315937798341], [-0.31896600469180014, 39.44312820995962], [-0.31898752834357114, 39.443097175818444], [-0.31900290236849893, 39.443075008571796], [-0.31903309172349426, 39.44302615681006], [-0.3191031952209308, 39.442883633820536], [-0.3191469325292697, 39.44280357290165], [-0.31917005063399206, 39.44275005362438], [-0.3192522799574252, 39.442594297089876], [-0.3193028864832732, 39.44250537677954], [-0.31935149440362803, 39.44241191498323], [-0.3194100696987616, 39.44230516090609], [-0.3194581597510878, 39.44222519123392], [-0.3195289812170966, 39.442082684511355], [-0.3196387491232161, 39.4419095542642], [-0.3197175123173374, 39.44184380494878], [-0.3197932415328318, 39.44180050749606], [-0.31996038610178573, 39.44175931789489], [-0.3200412581583713, 39.44175217387179], [-0.32014265829144045, 39.44175901622972], [-0.3202420896517144, 39.441779326237125], [-0.32032431056508226, 39.44181275250392], [-0.3204208697421132, 39.44185100463855], [-0.3207279299279543, 39.44194817078389], [-0.3209103032510732, 39.44200191376217], [-0.3210277361211695, 39.442045159919104], [-0.32137580247346476, 39.442152269254464], [-0.3220064500718826, 39.44235147190304], [-0.32218519174184795, 39.44240513822999], [-0.3222431029671208, 39.442428994322704], [-0.32244399766747806, 39.442492170372915], [-0.32277974041861235, 39.442598991640594], [-0.3231260162483317, 39.44271507229881], [-0.3235622898883851, 39.44285123287547], [-0.3236310791781177, 39.442875338774684], [-0.3258627080337743, 39.443534759509575], [-0.3266618791138862, 39.443999068312706], [-0.3269995609095779, 39.444340149845374], [-0.3271284877332498, 39.44450076810732], [-0.32790475211008063, 39.444140240137976], [-0.3280203471394654, 39.44407983628675], [-0.3280634615233225, 39.444053801153025], [-0.3280065167439662, 39.44396690882311], [-0.3276537614709467, 39.443620979219055], [-0.3275030134173792, 39.44351841673182], [-0.32736707810980126, 39.44338916838247], [-0.3270920903697025, 39.4431936698673], [-0.326671751098014, 39.442963296985155], [-0.32638892731897595, 39.442839686682575], [-0.32576676490581025, 39.4426271897737], [-0.3255157683210074, 39.442544849069655], [-0.3209049477472247, 39.44110094193336], [-0.3208013070595428, 39.44105801358623], [-0.32072958869390367, 39.441015830014344], [-0.3206943483987912, 39.440987991408214], [-0.32066291404776776, 39.4409557361488], [-0.32062873862854063, 39.44091891331612], [-0.3206143714834273, 39.44089606023425], [-0.32059654354789924, 39.44086862299362], [-0.3205816292752688, 39.440841252917885], [-0.32055443614091245, 39.440755051882356], [-0.3205502145711634, 39.440732432676626], [-0.3205475871161974, 39.4406873283331], [-0.32054987146513575, 39.4406468415979], [-0.3205564972490272, 39.44060645494473], [-0.3205606598757603, 39.44059303777273], [-0.320570573872089, 39.44056173567552], [-0.320587184174664, 39.44052608356677], [-0.3218763361683516, 39.438096466697026], [-0.32800173939738947, 39.44008863434917], [-0.3304143391609785, 39.43563529818391], [-0.3301110963269773, 39.4355337410476], [-0.3302968317236006, 39.435195682742396], [-0.31487656883704573, 39.43016970658072], [-0.314686623541827, 39.430521155630906], [-0.3144033930558893, 39.43042902894547], [-0.31432942075529346, 39.43053992932509], [-0.3141695644190325, 39.43048668758379], [-0.3142380675753875, 39.43036666119012], [-0.3142561407398044, 39.430331043748176], [-0.31412243878449003, 39.43029642376909], [-0.31394267373081036, 39.43026974796992], [-0.3138373519738555, 39.43028983602225], [-0.31372813042375913, 39.430354877641186], [-0.3136379976223056, 39.43045189096301], [-0.31336087451714645, 39.430729252911014], [-0.3126283318673676, 39.43146454117843], [-0.31198901359921694, 39.43211638829681], [-0.3115733808914983, 39.4325211695028], [-0.31151890057188525, 39.432578466303035], [-0.3145131073438904, 39.434976354885244], [-0.3145773204054237, 39.434986847199056], [-0.3147020315533984, 39.43504828528513], [-0.3147534553282564, 39.435108030181034], [-0.31477445003538235, 39.43518508959924], [-0.3147535816445139, 39.435274686021394], [-0.3146822771582676, 39.43535411750009], [-0.3145755645345171, 39.435410209003415], [-0.3144796717531159, 39.435430515383054], [-0.3143516070658072, 39.43541854770783], [-0.3143184092791126, 39.43541327627688], [-0.31425126615927085, 39.4353846986095], [-0.31420242646770086, 39.43535203946165], [-0.3097744511794401, 39.43179942366344], [-0.30940603492703617, 39.43143055535229], [-0.3091812521264814, 39.431065011002886], [-0.309066277311237, 39.43075155648814], [-0.3090165624116187, 39.43042159518431], [-0.30902836885102936, 39.43013359749473], [-0.3091041408919733, 39.429806541218845], [-0.3091904624429463, 39.429601347232804], [-0.3124833494231428, 39.423515679814905], [-0.3126086748997647, 39.423333897567915], [-0.3127026047159545, 39.42323246826271], [-0.31275527947501713, 39.42318414677637], [-0.3128885375749192, 39.423079122088026], [-0.31301550167765474, 39.42300548243357], [-0.3131752836513999, 39.4229280967886], [-0.3133468827708543, 39.42286449720859], [-0.313525589786506, 39.4228235835442], [-0.3137373038977194, 39.422792441126965], [-0.31397862847691904, 39.42278450444248], [-0.31413930361222847, 39.42279722577069], [-0.3144001291944462, 39.4228482957733], [-0.3145006369683164, 39.42287764407167], [-0.3192276662322093, 39.42441907561377], [-0.32338285001810985, 39.42578950597672], [-0.32358072457335696, 39.42581658403972], [-0.3236842419688873, 39.425805454258715], [-0.3238033000142152, 39.4257676558388], [-0.3239300555364635, 39.425680486224955], [-0.3240209982682192, 39.42554295277089], [-0.32419371920861656, 39.425222621768874], [-0.32449524206159247, 39.42466652245002], [-0.3249331508630772, 39.42439733751635], [-0.3251417614627699, 39.42435258970052], [-0.32520596678086233, 39.424363075877835], [-0.32532776811898145, 39.42442443556783], [-0.32538502797890984, 39.4245023186882], [-0.32539315955162457, 39.424574576034374], [-0.32536429272917683, 39.42464598209299], [-0.3253076480187074, 39.42470323582257], [-0.32524479431714015, 39.42473332031246], [-0.3252206799347067, 39.42473726982974], [-0.32510410806866047, 39.42474810060572], [-0.3248722905691564, 39.424886897458336], [-0.32438467957755973, 39.42579004798606], [-0.3242081647158386, 39.42611479629822], [-0.3242766097332798, 39.42624248569768], [-0.32889862417004645, 39.427731561784974], [-0.3294578844081613, 39.427478661542914], [-0.3294572688802268, 39.42709578310503], [-0.3294945485763882, 39.42705159564363], [-0.329549977100403, 39.42702584267589], [-0.32964354433329796, 39.427027992177834], [-0.3297145345985092, 39.427070162502595], [-0.3297338137265332, 39.42709763173013], [-0.32980929495028843, 39.42914880073634], [-0.3298538715899727, 39.430374982921215], [-0.32988289963596845, 39.43047023290999], [-0.3299590206389789, 39.43062513071808], [-0.3300210419873079, 39.430712129751555], [-0.3301489620950428, 39.430841190896146], [-0.33027173509085844, 39.43093408955575], [-0.33040899581653344, 39.43100931226806], [-0.33056398730249714, 39.43107593322993], [-0.3306930157079062, 39.43111942681349], [-0.3315798470961803, 39.43141454745752], [-0.33271183871099475, 39.43179185291555], [-0.33277132379335345, 39.431793208878354], [-0.3328732879850123, 39.431727991659564], [-0.3328909971764815, 39.43170137165158], [-0.33330987124783895, 39.43168394666459], [-0.3333055797908209, 39.43162530012577], [-0.3333042561383876, 39.431602747799424], [-0.33288757657414025, 39.43162021411623], [-0.3328830867657349, 39.43147147518005], [-0.33286260328372297, 39.431475509572714], [-0.3328035007916219, 39.430095845486264], [-0.3306681057330096, 39.430118916085476], [-0.3306131029789435, 39.43009513103142], [-0.3305965963207087, 39.43009024757171], [-0.33057606932979533, 39.43007626300158], [-0.33055226483047234, 39.43005319437794], [-0.3305424322851912, 39.43002594222711], [-0.33054073362991143, 39.42999437248066], [-0.3305553570447386, 39.42987760315117], [-0.3306471972402635, 39.4298301639218], [-0.33082943920433977, 39.42982984442696], [-0.3308537592176245, 39.42983941165822], [-0.330872835395127, 39.42985336286559], [-0.330896189375182, 39.42992596037477], [-0.33172903620053873, 39.4299045417415], [-0.3327625090196887, 39.429401255448234], [-0.33286910405183434, 39.429385683746446], [-0.3328678094156277, 39.42928655739471], [-0.3330953681810674, 39.42927826532252], [-0.3330936355052969, 39.42922867724422], [-0.33286752769610994, 39.42923700260701], [-0.33284734134644234, 39.42894826727735], [-0.3327927589353181, 39.428951528267625], [-0.33278815768450565, 39.42891989193678], [-0.3328441905153006, 39.42891667323723], [-0.33283843350884873, 39.42876339176983], [-0.33278257250096366, 39.42876211001731], [-0.33277965512401936, 39.42866745982508], [-0.33283277458056865, 39.42866417427942], [-0.3328206563509055, 39.42833508450781], [-0.33272535469892794, 39.42828334942101], [-0.3302722441599808, 39.42835767068874], [-0.33026987789731177, 39.42826753751538], [-0.3327184615912503, 39.42819760777329], [-0.3328099219475477, 39.42814114927731], [-0.33275462623956725, 39.42702282681031], [-0.33269787535953266, 39.427026029007756], [-0.3326981859551362, 39.42699900976886], [-0.33275565638407495, 39.42699582408148], [-0.33275170883994154, 39.42692817659582], [-0.3306355955830441, 39.42614542175726], [-0.3296899369362859, 39.42604263200625], [-0.3297008900156533, 39.426344668918226], [-0.32964477454991525, 39.42638842375644], [-0.32958460733805794, 39.42640505012307], [-0.3295555923580825, 39.426404392572465], [-0.3294993576131923, 39.426394091889634], [-0.32944090902422163, 39.42636572274929], [-0.3294225652268746, 39.426351788139485], [-0.3294100871040794, 39.426013681037624], [-0.32658439476270307, 39.42571452718636], [-0.3265551492105832, 39.425871499241744], [-0.3265265959008079, 39.42591588645132], [-0.3265009011755265, 39.42594232183658], [-0.3264695756428532, 39.42596412331927], [-0.3264275359837935, 39.4259811739789], [-0.3264003592934519, 39.42598955769412], [-0.326352141360232, 39.42599745744363], [-0.32631932226946375, 39.42600120697059], [-0.32624658304562776, 39.42598602071043], [-0.3262315167954835, 39.425981169772676], [-0.32616110048130836, 39.42594351493861], [-0.3261282075742925, 39.42591122759155], [-0.3261118316323165, 39.425883824560835], [-0.3260979814596372, 39.425847479855726], [-0.3260937587107457, 39.42582486076919], [-0.3260967584319696, 39.425784390258166], [-0.3261765769752043, 39.42561505944096], [-0.32621021365840125, 39.425570789243444], [-0.32623030875296316, 39.42555773829246], [-0.32624153908474723, 39.425548987826126], [-0.3262975265570425, 39.425527753654336], [-0.3263210821044396, 39.425519286671694], [-0.32634953723297716, 39.42551543676807], [-0.3263618740671324, 39.42551572052135], [-0.33050914107114554, 39.42593982913103], [-0.3306016440642069, 39.42595095318294], [-0.3306774692246449, 39.42596170319284], [-0.3307665550515875, 39.42598627083374], [-0.33082098374456886, 39.426005538217844], [-0.33297503060875383, 39.42680267319217], [-0.33301730673948027, 39.42681715633792], [-0.3330492231177306, 39.42681788860073], [-0.3330682544657728, 39.42681382084126], [-0.3330957741056485, 39.426796434638526], [-0.3331361458515505, 39.426765830102056], [-0.33317583091391323, 39.426753227376395], [-0.333249270914079, 39.42675040781361], [-0.33327231115174494, 39.42675544077912], [-0.33329862914039776, 39.42676955772438], [-0.3338329137486058, 39.42697999783095], [-0.3339693951140618, 39.427037180871366], [-0.3341606701343338, 39.42676229511663], [-0.33561007337999127, 39.42728648706856], [-0.3360548430672957, 39.426341775701864], [-0.33604144810963954, 39.42742653286556], [-0.33593007086210463, 39.42745887139538], [-0.33584731728481826, 39.42757805296476], [-0.3357390426384211, 39.427683208412354], [-0.33564300813326814, 39.427923172596394], [-0.3355387644215719, 39.428378213580565], [-0.3353628078748986, 39.430018284332306], [-0.33540775354569424, 39.42997894625712], [-0.3354501765824387, 39.429778121846546], [-0.335514779067109676, 39.42967869524948], [-0.3356685143426788, 39.42952077258327], [-0.335952472780999, 39.429365834283495], [-0.33612304645419977, 39.42931379884607], [-0.3362337030603991, 39.43187258767346], [-0.33653051991425637, 39.434978303628945], [-0.33663085640158213, 39.43602108122877], [-0.3365901589602382, 39.436916495543166], [-0.33614136599562905, 39.44068527674086], [-0.33590558110330476, 39.442941805833044], [-0.33578694945991766, 39.444120104321385], [-0.33573317756865817, 39.44495049397096], [-0.33554765986427876, 39.44655880204759], [-0.33546644121849245, 39.447412764090636], [-0.33528264516616757, 39.448404019085324], [-0.3356094141689782, 39.448443032175], [-0.3354265760653058, 39.44944782236034], [-0.33522872746540733, 39.45065495607583], [-0.33490574889835895, 39.452629468973015], [-0.3348932817404425, 39.45383184351337], [-0.33459128413740125, 39.45477984627252], [-0.3416905851127714, 39.45410730797738], [-0.3408311732735019, 39.45784054144553], [-0.3406208545537822, 39.458251142837696], [-0.3400701143204393, 39.46006730839146], [-0.33989588380133157, 39.460446192758546], [-0.33984515142917315, 39.46055764099684], [-0.33943086648697646, 39.46107967406734], [-0.3385419512376385, 39.46176200331785], [-0.33775246540844217, 39.46237002887816], [-0.33701443970081696, 39.46303779394601], [-0.3364954890030063, 39.46367453129206], [-0.33542506438542136, 39.46595171897474], [-0.3349802995066727, 39.46854502895326], [-0.33848083681675945, 39.468814410072206], [-0.3386762911994794, 39.46883239946263], [-0.3387565001137917, 39.46867207886738], [-0.33986199241962994, 39.46899917376961], [-0.3397985769671237, 39.469119340209986], [-0.3400979665394179, 39.46921177637533], [-0.3398176366243958, 39.46978191040689], [-0.34023995032491927, 39.469908690678594], [-0.3401425746792323, 39.47010014998701], [-0.3394431790352495, 39.4698859563001], [-0.33877609701106115, 39.471244506296856], [-0.3382061554914052, 39.47107284140114], [-0.33776861036885447, 39.47194594445745], [-0.3374358106716363, 39.472686675806095], [-0.33710449731799874, 39.47365737824093], [-0.33550869496966856, 39.476998491255486], [-0.33338084741316704, 39.47641315451994], [-0.3328155240447492, 39.47918884063302], [-0.3325424508959938, 39.48046164548878], [-0.3315104329274244, 39.48560696385567]]],  "type": "Polygon"}
"""

# 2. POBLATS DEL SUD
# Pegar aquí el JSON para Poblats del Sud si no está definido
POLY_SUD = """
{"coordinates": [[[-0.4143525635090557, 39.459593381039404], [-0.409208497385736, 39.45230965419426], [-0.40585349728372644, 39.44764481340107], [-0.40331934441065476, 39.444899080072744], [-0.40107312319346644, 39.44281289160304], [-0.39802887840930645, 39.4404385693215], [-0.393007123216077, 39.438348638829254], [-0.3898332503218489, 39.43751167631327], [-0.3871549923651072, 39.43719024716469], [-0.3780915283821877, 39.43630148854651], [-0.37017371913482555, 39.43551903836273], [-0.3655553310381972, 39.435144192893894], [-0.36213498378531483, 39.43480987147675], [-0.3613896025630597, 39.43466683917187], [-0.35908200569736526, 39.4339207727042], [-0.35541385980599, 39.43269329181143], [-0.3511369660128623, 39.43126715407057], [-0.34896042927077286, 39.43053740610393], [-0.3460806275820897, 39.42953483055811], [-0.3376425337756836, 39.42657182115788], [-0.3368460442136396, 39.42636440040121], [-0.3360548430672957, 39.426341775701864], [-0.33502394785444195, 39.425047951189526], [-0.33418803532335034, 39.42188031849265], [-0.3335873081029165, 39.41955135188783], [-0.33314067178731754, 39.417027741336014], [-0.3308857185579097, 39.408201693770565], [-0.32809228731297746, 39.397660612122856], [-0.3264173500378982, 39.389577474728384], [-0.3230658965252253, 39.38010877773956], [-0.31917775335224946, 39.36885734673691], [-0.31703007109295767, 39.363852998533325], [-0.3123926867873943, 39.350795828907714], [-0.3060888479697947, 39.33667289354762], [-0.30070212436696153, 39.324755490801714], [-0.29206672086381064, 39.308262295752705], [-0.2858101010552789, 39.29644553352574], [-0.28010349182844635, 39.28697909452221], [-0.27539424910443927, 39.27914382530805], [-0.27536845690029316, 39.27899763779059], [-0.27608733634937765, 39.27892709184794], [-0.2764031073868175, 39.279077355548], [-0.27831937912699056, 39.27924381458721], [-0.27878823155686977, 39.27913369204001], [-0.280132918422762, 39.27940743535074], [-0.2814155456209547, 39.27973564944946], [-0.2829358447295629, 39.280242082170915], [-0.2845756250437791, 39.2808263727816], [-0.28602024356352873, 39.28113973148193], [-0.28673321733014356, 39.281316159179184], [-0.28818630259758815, 39.28176840761571], [-0.2905703435178986, 39.28260965794733], [-0.2916886717211176, 39.28294197864544], [-0.2912541230636598, 39.283544477479474], [-0.2915950494828362, 39.28359235930512], [-0.2928234498729032, 39.28388752015955], [-0.2933564675829166, 39.28443822962193], [-0.29451390201110317, 39.28586478093364], [-0.29616254012512744, 39.287563625778354], [-0.29865490154280877, 39.2903131181848], [-0.30066942943234765, 39.292647360781466], [-0.30224922857242054, 39.29450118893253], [-0.30310891939881, 39.29576108748325], [-0.3038927193787861, 39.29684819612326], [-0.3054673282151234, 39.299247143196546], [-0.30646332101630996, 39.29982603799115], [-0.3077576689484037, 39.30051873766489], [-0.309098068996175, 39.30144764780998], [-0.3103977257447323, 39.302373763039014], [-0.3115580178724753, 39.30306262814976], [-0.312297768765119, 39.30346522065321], [-0.31269706712942974, 39.303559966261076], [-0.3140003092194337, 39.30361517924175], [-0.3147337885727799, 39.303900288564584], [-0.31516398126081296, 39.30393535208859], [-0.3155330134822801, 39.30387682358893], [-0.3158342706636287, 39.30361558820512], [-0.31626359602885656, 39.3033908375539], [-0.3167067594082446, 39.303259061142185], [-0.31772185915359796, 39.303110317895], [-0.3181415037139215, 39.3030817404119], [-0.3182477213025611, 39.30352408534453], [-0.31833442871556844, 39.30383210793942], [-0.3183659041897226, 39.30494211941384], [-0.31848503447330295, 39.30569077655666], [-0.3185245500896784, 39.30659059690101], [-0.3185901105668401, 39.30745278361096], [-0.3186608968461693, 39.30855347547993], [-0.3185655752632884, 39.30900586812911], [-0.3184762180530181, 39.31069326747229], [-0.31847669576503146, 39.31179290588602], [-0.3200417456451929, 39.31182357410999], [-0.3206385952439034, 39.31198253682717], [-0.3211177056730735, 39.31232163916824], [-0.3215633981566205, 39.31318839685593], [-0.32201129033388365, 39.313801080142795], [-0.3225467854522764, 39.31392970222658], [-0.32295251111666984, 39.31393903058945], [-0.3283440977403122, 39.31299570931909], [-0.3289058980429404, 39.31300860108573], [-0.3293060863887959, 39.31316337324952], [-0.3305562474084222, 39.313968474068545], [-0.3330052300599437, 39.31562631444278], [-0.3341938778834918, 39.31723065274749], [-0.33579054931346486, 39.314671047177285], [-0.33697842180357224, 39.31464969645789], [-0.33708992089173145, 39.316182226508886], [-0.3474878335550889, 39.31528803024235], [-0.3496785076638589, 39.31106614002623], [-0.3524577949046548, 39.31130875609524], [-0.3560056218654985, 39.31238904128462], [-0.35906641665828787, 39.30681891408959], [-0.3607252323219664, 39.30663454059528], [-0.36146211598598765, 39.3057494114905], [-0.36316605419512105, 39.30627996073925], [-0.36458715045896695, 39.30497305686436], [-0.3675741956735851, 39.30662153426816], [-0.36739361869672643, 39.30718694219038], [-0.36785092730077323, 39.30774201783359], [-0.375407896390661, 39.30731332441193], [-0.37603809161467316, 39.307562124964406], [-0.3774292495544644, 39.31000347198964], [-0.3800105626048336, 39.31106560487089], [-0.3768036933512461, 39.31233197710126], [-0.38155948411649543, 39.31670601011369], [-0.38476139692009836, 39.31750312437739], [-0.38508019305485225, 39.31626061655984], [-0.38997639671528395, 39.31494826341362], [-0.39069598509498055, 39.31490862169756], [-0.3917026478711374, 39.32306736345644], [-0.39443626792697123, 39.323211930382], [-0.39672937590761204, 39.32940091409979], [-0.39842319914070295, 39.32931627733559], [-0.3987359194789071, 39.33188916037018], [-0.3962928818960573, 39.33211458251682], [-0.39632169558294156, 39.332552857420445], [-0.39656343736615246, 39.333328474181755], [-0.3968040574758966, 39.333526408628565], [-0.3978073081446849, 39.333828946261825], [-0.39763406169210885, 39.33424517788655], [-0.3968289443553111, 39.335592394335876], [-0.3927983965557545, 39.334854457131435], [-0.3924325422133205, 39.33559895698303], [-0.3946190545847701, 39.336181167539046], [-0.39435177125505416, 39.336700323283864], [-0.3930621332290472, 39.33803681037969], [-0.391052250589742, 39.33849021346619], [-0.3906948388259811, 39.33900733863311], [-0.39079510403127443, 39.33934217230573], [-0.39094172941081845, 39.33964304591888], [-0.3905033147787493, 39.339913300143856], [-0.389575145640145, 39.34001500425339], [-0.38799734457988766, 39.340609769992774], [-0.3882233734734433, 39.34297180883661], [-0.3867362443873755, 39.34355107890481], [-0.38659442361921453, 39.34632288808761], [-0.38643274386300885, 39.347502510347475], [-0.38612102248857727, 39.34818678087034], [-0.38625188374180836, 39.349343994806866], [-0.3858862864177247, 39.35061959084379], [-0.3862711262847222, 39.35181328721429], [-0.3852948179070383, 39.352408555005624], [-0.38516611449668325, 39.353201989952524], [-0.38273214765827746, 39.354505125015244], [-0.3826229616493433, 39.35548600349861], [-0.38157841252317637, 39.356203161625885], [-0.38073396132952786, 39.35624585773562], [-0.3800714360484429, 39.356527211932054], [-0.379116428955781, 39.35697478093969], [-0.3782778384237938, 39.35759890190988], [-0.3771881125657257, 39.35782122127394], [-0.375146278056673, 39.35802713537654], [-0.37193729377760426, 39.35899001601747], [-0.3702063398618125, 39.36249777282596], [-0.3636161310722, 39.359753679031], [-0.3606556529392181, 39.362395796435344], [-0.36099274208984805, 39.36387880793187], [-0.35782557580711677, 39.36440650804211], [-0.3539101632592203, 39.365446650004074], [-0.35345703280424395, 39.365845171254115], [-0.3524769245730385, 39.36665708534187], [-0.3513556797360041, 39.367410164487595], [-0.3517355802993438, 39.36800238527154], [-0.34709045113494064, 39.36995351575404], [-0.3441142823043357, 39.366602251465096], [-0.3437561526759774, 39.36593636063423], [-0.3411620743060766, 39.366826569817015], [-0.34317024743152286, 39.37081873233296], [-0.3370125027869383, 39.373018753998906], [-0.3342759701156975, 39.36825392457544], [-0.33241951780187873, 39.368132913527774], [-0.3309846389275196, 39.36437914113543], [-0.32990536059055714, 39.36448857053098], [-0.32999424668234, 39.36497017101871], [-0.3301050174041461, 39.3655579027484], [-0.3302425649795477, 39.36614928507111], [-0.33036796518394174, 39.36658586840152], [-0.330499497894156, 39.367100050528805], [-0.33066609972384936, 39.36762811834781], [-0.3308029800247862, 39.368230214172144], [-0.3309718743844331, 39.36890021463666], [-0.33110745567997035, 39.36949356867699], [-0.3312324297948138, 39.36995958216601], [-0.3313429829742034, 39.370379157242844], [-0.3314763195908272, 39.37082376813349], [-0.331643664325987, 39.37143723743946], [-0.33171656725912174, 39.371806876950735], [-0.33182713793805113, 39.372226433910704], [-0.33192624159600276, 39.37275266274711], [-0.33210054255235755, 39.37322757261551], [-0.33231200531430816, 39.37386115184101], [-0.33244911537707555, 39.37430817239936], [-0.3325933651399241, 39.374698510708185], [-0.3327641063680753, 39.37518726558864], [-0.33287633261093724, 39.375481618408116], [-0.33299739691557095, 39.37589415374095], [-0.33311042588093126, 39.37616838995833], [-0.3332387425136259, 39.37661519886839], [-0.33333937393145124, 39.376979437238056], [-0.33342879971808403, 39.37729319420707], [-0.3335260659049816, 39.37763122956688], [-0.33362443188599183, 39.37794832646588], [-0.33373579812507903, 39.37837767433988], [-0.3338333039042712, 39.378712841097006], [-0.3339307223601706, 39.37893017903127], [-0.33401812428551597, 39.379186529778174], [-0.3341511143793245, 39.37951052804549], [-0.3342547480707525, 39.37968021625841], [-0.33442411658728066, 39.37985823099405], [-0.33459839463698127, 39.380138770615645], [-0.334806089408488, 39.38049753399669], [-0.3350256540593718, 39.3808610734811], [-0.33516163731647336, 39.381106609076824], [-0.33544199507371497, 39.381613358296015], [-0.3356622903127819, 39.38207319990234], [-0.3358269953532122, 39.3823740315051], [-0.3360868431938564, 39.382747293629556], [-0.3363351514062996, 39.38316610983303], [-0.3365355419023349, 39.38358153966123], [-0.3367534585998913, 39.38399437063014], [-0.337001786141748, 39.384413194882065], [-0.33722157135783287, 39.38477732991921], [-0.33744078376581405, 39.385151090896734], [-0.33755142355383216, 39.38525264959374], [-0.3376027513262989, 39.38528736480985], [-0.33773724550096457, 39.38537831658089], [-0.3379156992968787, 39.38554970564915], [-0.33834676912278905, 39.385970377478976], [-0.338585492104789, 39.386169811334064], [-0.3389483483886687, 39.386468408271554], [-0.33936761749228456, 39.38683881622517], [-0.33960874572427363, 39.38707042860431], [-0.33974102468224415, 39.38717920096637], [-0.33993946674530107, 39.38734236878169], [-0.3401218556704314, 39.387505178042886], [-0.3404636080783947, 39.387782360011066], [-0.3406859619325701, 39.387950920269226], [-0.3408594674446517, 39.38808788603935], [-0.3410499362108348, 39.38826854500516], [-0.3413873054985016, 39.38860875807881], [-0.3416149052637679, 39.38882732754115], [-0.3418848162349607, 39.3891757624487], [-0.3420820624121976, 39.38944073535181], [-0.34236850081758097, 39.38980471786444], [-0.34256244069401093, 39.39006482168927], [-0.3428587131199384, 39.39038671321997], [-0.343101875866722, 39.39066658935132], [-0.34339736101474055, 39.39098090312325], [-0.34366089377647285, 39.39127634235638], [-0.34390377368181707, 39.39157607489139], [-0.3441939614720403, 39.39192983252504], [-0.3443809845764626, 39.392201811206604], [-0.34449562626625035, 39.39239950716015], [-0.34456774284557645, 39.39256222340131], [-0.344671727925958, 39.392758369519115], [-0.34477643361398264, 39.39293583864305], [-0.34491510974687317, 39.39312959664911], [-0.34496990062128846, 39.39327467406548], [-0.3450062498237985, 39.39336415115561], [-0.34508166690240005, 39.39356847323174], [-0.3451783749222756, 39.39383778474875], [-0.3452414139899493, 39.394010013847975], [-0.3452983741532908, 39.39411660079232], [-0.3453855091996379, 39.39424220965993], [-0.3455148139960753, 39.39439600596691], [-0.3456319147872145, 39.39458995522423], [-0.3457356224017783, 39.39478622019077], [-0.34584514656826576, 39.39496260919756], [-0.34595730599798213, 39.39514116632792], [-0.3461399515280035, 39.39547570783555], [-0.3462178694723547, 39.395630438509116], [-0.3463268521166674, 39.39582148111054], [-0.34645307829069355, 39.396056916361744], [-0.34652229908403115, 39.39622945702266], [-0.34659736886274783, 39.396388302547166], [-0.3467279780187654, 39.396590865154224], [-0.34684624907954337, 39.39676269617008], [-0.3469584860386929, 39.39692157879988], [-0.3470547676264692, 39.39702935944006], [-0.34717180887941423, 39.39715800969896], [-0.34730540306534285, 39.39728587536823], [-0.34742738555141045, 39.39740794453982], [-0.34757507410231403, 39.39753342880792], [-0.3476862664905976, 39.39761761259648], [-0.3477843181803821, 39.397690992206826], [-0.34788186049215647, 39.397777206730886], [-0.348044937144005, 39.39792961752146], [-0.34819859215117716, 39.39804509314208], [-0.3483036650760493, 39.39813147006831], [-0.34838754039931574, 39.39819566115057], [-0.34865126132842744, 39.398345150280136], [-0.34874542030288075, 39.39840210728484], [-0.3488033283829973, 39.398467958139], [-0.3488355397314039, 39.39853168250133], [-0.3488709954587988, 39.39863237205988], [-0.34891279771615435, 39.39876047609867], [-0.34897828977337997, 39.398858787501894], [-0.3490397433576887, 39.39893558375179], [-0.34913522477876513, 39.39906157836263], [-0.3492421573923571, 39.39918200525231], [-0.34934877791378, 39.39930546992038], [-0.3495079945667364, 39.39950570874903], [-0.34963705201928663, 39.3996879808939], [-0.3497035593827936, 39.39976106324568], [-0.34991060077667385, 39.39992986977197], [-0.3501797566494397, 39.4001617842031], [-0.35042497681019896, 39.40034385614057], [-0.35053737199364493, 39.400460883890965], [-0.35058471987795964, 39.40059075332695], [-0.35066373221332653, 39.400778622537615], [-0.35072048491032426, 39.40090392252998], [-0.3507904435750493, 39.401095702392276], [-0.35090222422680584, 39.401391586429924], [-0.3510180796414413, 39.4017399766628], [-0.3510739424647278, 39.40190293119798], [-0.3511410063488252, 39.40216203105251], [-0.3511404572577374, 39.40233851082632], [-0.3511379508563687, 39.4024863249018], [-0.3511152835572225, 39.402678985116204], [-0.35109556326377644, 39.402797164277374], [-0.3510667346035306, 39.40298969310074], [-0.351038560692989, 39.403165534431835], [-0.35101979064593, 39.40333682433561], [-0.35099871214084266, 39.40347369289618], [-0.3509711253564366, 39.40381222926031], [-0.35093806725433757, 39.404116758676686], [-0.35089907834006195, 39.40440984683292], [-0.35086087886336736, 39.404765807637254], [-0.3508155509866729, 39.40515115497435], [-0.3508140198322688, 39.40535384603234], [-0.35080587407499053, 39.40565026748632], [-0.3508138606443731, 39.40592638124359], [-0.3508174106150746, 39.406319977271025], [-0.35082780733377583, 39.40670327003089], [-0.3508090308006343, 39.406955648213426], [-0.35079934165881993, 39.40728790762588], [-0.350794476618314, 39.407497666662756], [-0.3507869680111436, 39.40769167205789], [-0.3507994501771844, 39.40784935890653], [-0.3508023467117051, 39.408016376689794], [-0.3507858955835218, 39.40812022505442], [-0.350769558870172, 39.408227184070824], [-0.3507462373879375, 39.40835544296599], [-0.3507228374270196, 39.40848607839788], [-0.3506920470495027, 39.40864995017379], [-0.35066512107532044, 39.40879242392545], [-0.3506406780566039, 39.40894262076991], [-0.3506189814210179, 39.4090540250825], [-0.3505646469054541, 39.40933421386562], [-0.3505261298796348, 39.40954354824379], [-0.3505175216572966, 39.40965524179721], [-0.3505045502182962, 39.40976684491912], [-0.3504786074471502, 39.40987474707855], [-0.35045600065315363, 39.41000985986824], [-0.3504253092745204, 39.410127842778955], [-0.35039757030918495, 39.41028316251429], [-0.3503885027209574, 39.41040360216091], [-0.3503529242840015, 39.410650299984326], [-0.35031928134050205, 39.41084619604359], [-0.350277204900893, 39.411026197472296], [-0.35023724531935246, 39.41121898561719], [-0.35018584396678837, 39.41140968415594], [-0.3501417158254775, 39.41164168277339], [-0.350087139119257, 39.41193673024335], [-0.3500718247749169, 39.412086540373664], [-0.3500577424790097, 39.41221915369624], [-0.3500518422526087, 39.41231635061816], [-0.3500340580401264, 39.41250314874809], [-0.35002471127687096, 39.41266740991811], [-0.3499863213292679, 39.412856729259815], [-0.34996176651108224, 39.413030940839725], [-0.3499339971796462, 39.413212745647016], [-0.3498977616019682, 39.413382889206765], [-0.3498631784782058, 39.41353454826552], [-0.34984474725890624, 39.41364367546899], [-0.3498317220116074, 39.413751736738824], [-0.34981178258059314, 39.413921819322915], [-0.34979607352658515, 39.41406208902635], [-0.34978022363912176, 39.414245003307904], [-0.3497381557220907, 39.41456506486552], [-0.34971310179503134, 39.414913027229076], [-0.34964197942378783, 39.41541379204195], [-0.3496202176305879, 39.41555622995277], [-0.3496136363398207, 39.41565175364729], [-0.3496285213535534, 39.41573922637379], [-0.34965627804540345, 39.41583091138982], [-0.3496708071274628, 39.4159568977363], [-0.349688567519698, 39.416035922665586], [-0.34970196645917767, 39.41615556805366], [-0.34971337416644527, 39.41629772614303], [-0.34972532007263, 39.41642414907142], [-0.3497368027408169, 39.41668969373758], [-0.34974817607293884, 39.416831842022475], [-0.34973975374407223, 39.416928134531126], [-0.34973526962392903, 39.41707283114861], [-0.3497362829675387, 39.417271985315125], [-0.34975224246117786, 39.417545055284414], [-0.3497707455226148, 39.417753454828436], [-0.3497777743424292, 39.41794619670071], [-0.34980200568526026, 39.41813737582665], [-0.3498198843623603, 39.41837267947981], [-0.34984409591301435, 39.41859565027499], [-0.34985819203450047, 39.418899821210005], [-0.34990180183324515, 39.41929900533909], [-0.3499310497056724, 39.41953712663036], [-0.34994608201833766, 39.419749473924256], [-0.3499540853634875, 39.41991712136175], [-0.34996704319259375, 39.420103602070284], [-0.349972133965138, 39.42037048734127], [-0.3500117918383744, 39.42067597962812], [-0.35003742274045846, 39.42100527775931], [-0.35004493142595755, 39.42130276702595], [-0.3500501281307514, 39.42141605428781], [-0.3500662973085265, 39.421493583419014], [-0.3500883194310817, 39.42161313698084], [-0.3501051768694418, 39.42171787041297], [-0.35012686552785655, 39.4218462269896], [-0.3501541101282332, 39.42198705229102], [-0.3501661580615031, 39.422130738319645], [-0.3501774078804209, 39.42223387547402], [-0.35038656461478374, 39.42221153542541], [-0.3506001860196253, 39.42218325183493], [-0.35078270103442366, 39.42215957415709], [-0.3508573694116896, 39.422149330094946], [-0.35100919335454683, 39.422127574206165], [-0.3511145509057455, 39.4221160653827], [-0.3512199170695913, 39.422107088138105], [-0.3513883046925273, 39.422092375693396], [-0.3516840324802233, 39.422071501231684], [-0.35199877027589804, 39.422055644536066], [-0.35236061316866885, 39.422031373566575], [-0.35265623786949074, 39.422006152020224], [-0.3530424316141146, 39.42198449648803], [-0.3532454630134585, 39.42197020081369], [-0.353330158947533, 39.42196475994635], [-0.35341993024683255, 39.42196516417909], [-0.35350437885566954, 39.421966257960015], [-0.35356536845206543, 39.421973385057946], [-0.3536027829301851, 39.42193712946449], [-0.35380252885785995, 39.421795706560644], [-0.35413569931673494, 39.42156800669921], [-0.3543721781711372, 39.42139997795487], [-0.3547372574312429, 39.421135282544746], [-0.35508212913232307, 39.42087166668897], [-0.3554431215727764, 39.42062520896117], [-0.3557356852388293, 39.42042886821824], [-0.3558544790719092, 39.42034360810124], [-0.356106573155161, 39.42016079604436], [-0.35642790133568186, 39.41992579316114], [-0.35662119913326873, 39.419792407634226], [-0.35692082499815614, 39.41960017926965], [-0.3571030797762941, 39.41950017180817], [-0.35722389998927095, 39.419553917555255], [-0.3574306131985054, 39.419646696074324], [-0.35767783423734967, 39.41972026022875], [-0.3579683523641574, 39.41976995252296], [-0.3580830912285279, 39.419784550924355], [-0.35832030435404627, 39.41976824838211], [-0.3588254951171106, 39.419606559536774], [-0.35895425897627037, 39.419597629335065], [-0.3592954598529692, 39.41963147007739], [-0.3595722520367514, 39.41967095486657], [-0.35982716827631833, 39.41969088848263], [-0.36000068742230595, 39.41971142346462], [-0.3600809287964545, 39.41969190376494], [-0.3602826554616887, 39.419613495207585], [-0.36044933172929977, 39.41957221765391], [-0.36084075173686087, 39.419466755346505], [-0.3611708833143046, 39.41938887292085], [-0.3615227594256518, 39.419302005777006], [-0.3618462682318266, 39.41924292569516], [-0.3620971182521815, 39.41929796858345], [-0.36237637071977, 39.41931780919174], [-0.3626300388721975, 39.41934064520972], [-0.36274863333809, 39.419375596364986], [-0.36284012427306156, 39.41941563514931], [-0.36313282005411507, 39.41952685829081], [-0.3633441601965065, 39.419606182015045], [-0.36363860748344146, 39.419715353569956], [-0.36383193495843025, 39.419780475379476], [-0.36402933264262416, 39.419831608342825], [-0.3641430464705543, 39.41986388890738], [-0.3642249489742265, 39.41988594376704], [-0.3642954517230658, 39.419901055544905], [-0.3643889214374235, 39.41992438154035], [-0.3644443283836095, 39.419905493940924], [-0.3645325716345212, 39.41992234108657], [-0.3646778789631676, 39.41996632816272], [-0.36478676172576413, 39.42000939027438], [-0.3648240303085728, 39.42003200045774], [-0.36483945563520515, 39.420055412759176], [-0.36483757157978447, 39.4200770543355], [-0.36477547298382557, 39.4203339391804], [-0.3647016646999962, 39.4205782164139], [-0.3646471919350355, 39.42078250925057], [-0.3646208713191319, 39.42086050569501], [-0.36460702269387324, 39.42091761409885], [-0.364614012768823, 39.4209494206249], [-0.36462523017470966, 39.420967989899516], [-0.3646424389505356, 39.42098075817092], [-0.36467611432406577, 39.420997016792406], [-0.3647225169627823, 39.42101212250822], [-0.36481833078824466, 39.421042330013165], [-0.3649191027417236, 39.421076280391844], [-0.36497598146886595, 39.42109079466106], [-0.36508348155176323, 39.421119122737856], [-0.36519502023601447, 39.42115956905562], [-0.3652858411528452, 39.42118141992245], [-0.36535469825472694, 39.42120233150283], [-0.3654156387184712, 39.421225270791886], [-0.36550994612887167, 39.42125663275811], [-0.3655668275451193, 39.42128218263297], [-0.3656304377758649, 39.42130948853945], [-0.3656593202767063, 39.42132595357484], [-0.3656728744472529, 39.42134046767281], [-0.36568784415038286, 39.421357797578935], [-0.3657044448814384, 39.42137838960923], [-0.3657372214031002, 39.421401870667175], [-0.3657877558127425, 39.421430952215985], [-0.3658323757747125, 39.421441593755475], [-0.36589914285700365, 39.42147544840803], [-0.3660113193936912, 39.42152735864267], [-0.3662098355837387, 39.42161866571133], [-0.3664942073573563, 39.42174651141628], [-0.36669094993479034, 39.421832822629455], [-0.3669029372240358, 39.421922397681904], [-0.3672365727444376, 39.42206133923377], [-0.3674472256202182, 39.42216881068188], [-0.36771816916082645, 39.422298340284854], [-0.36791246802630206, 39.422395035369625], [-0.36806301683722464, 39.4224570375226], [-0.3681433355482387, 39.42249011665817], [-0.36845869794437386, 39.422658262117075], [-0.368707715115293, 39.4227766806656], [-0.36887335748908423, 39.42284445575939], [-0.3691262192370065, 39.422948095844546], [-0.36939284411884404, 39.42305796564851], [-0.36954455966562116, 39.42311866800582], [-0.36976871690142343, 39.42320327913199], [-0.3699817244075119, 39.423287871805584], [-0.37020471845848113, 39.42337039275714], [-0.37039222407093797, 39.42343817382803], [-0.3705346239270348, 39.42349953803541], [-0.37077086316474117, 39.42359590681663], [-0.3708325027428283, 39.42362568781369], [-0.37090868677868855, 39.423666238975905], [-0.3711405373174755, 39.42377061571975], [-0.37124689283758466, 39.423818443445406], [-0.3713798664410731, 39.423875062017764], [-0.3715319107665091, 39.42394565194806], [-0.3716641722874384, 39.42399866857654], [-0.3718146307384233, 39.42407778068993], [-0.3719778223010221, 39.424142315873716], [-0.37193114070516037, 39.42422508784882], [-0.37188301418126657, 39.42434568231264], [-0.37178470939565866, 39.42458674370168], [-0.371629284176618, 39.424964511000596], [-0.3715683393098697, 39.425097905335186], [-0.3714524255494281, 39.4253521715942], [-0.37138125057286486, 39.425509243967845], [-0.37137777715158804, 39.425564867114446], [-0.37139711466402386, 39.42560171813711], [-0.3714462101246428, 39.425655313828685], [-0.37152036047585824, 39.42568853947337], [-0.37158350029666803, 39.42571802068195], [-0.3716519321256889, 39.42574114415092], [-0.37172588519399885, 39.42577282469274], [-0.37184287975159414, 39.425820775326464], [-0.3719700282963862, 39.42587239678021], [-0.37206175571579747, 39.42591021757905], [-0.3721438816729221, 39.42595130767839], [-0.37222122056466517, 39.425988415703124], [-0.37228588408947955, 39.42602187685296], [-0.37233206577629235, 39.426045776217606], [-0.3724335581704294, 39.42610659182083], [-0.3725610916524462, 39.42620054489508], [-0.37263310067977046, 39.426235451137345], [-0.37276678409277975, 39.42630548036141], [-0.37285552901341984, 39.42633923320682], [-0.3729352409472457, 39.4263679260777], [-0.3730565781097131, 39.426402370285494], [-0.373130599383511, 39.42644061893419], [-0.3732522236505434, 39.42646773622618], [-0.37341031514032447, 39.426509587043675], [-0.3735347440615453, 39.426539974563184], [-0.37367595094459977, 39.4265724166615], [-0.3739721423763557, 39.426640071006375], [-0.37414361390311407, 39.42666126851348], [-0.37437215451734446, 39.42668837634155], [-0.37465004741084823, 39.42674098528698], [-0.37491921148304, 39.42679807204676], [-0.37508983580848126, 39.42681833913693], [-0.3752881364227346, 39.42685205047977], [-0.3755371538270712, 39.426876780583434], [-0.37566528392189363, 39.42688999744846], [-0.3757513064972758, 39.42690225458703], [-0.3758750643221128, 39.426927975897364], [-0.376024237037228, 39.426966117402934], [-0.3763474655297478, 39.427046718148674], [-0.3765835491356649, 39.42711618934441], [-0.37670369026302425, 39.42715287300679], [-0.37682615278302056, 39.42719796914398], [-0.37692475922030827, 39.427235229671446], [-0.3770602125425731, 39.427293546403654], [-0.37730078626845476, 39.427410891407405], [-0.37759183345122493, 39.4275493021609], [-0.3778881555556945, 39.42766820089232], [-0.3780285521512661, 39.4277353756084], [-0.3781859355514407, 39.427810203427704], [-0.3783492187157402, 39.427891272095295], [-0.3785802762347391, 39.427997949595], [-0.3787637166786392, 39.428100345762594], [-0.3789299538025689, 39.42820676808344], [-0.37907712421216555, 39.42831765208652], [-0.3793821743988417, 39.42852367927209], [-0.37967752583751113, 39.428718568314586], [-0.3797336753906865, 39.428734500415594], [-0.3799150249187546, 39.42885837881938], [-0.38009336015998235, 39.42899451311951], [-0.38034889022057755, 39.42922515099336], [-0.3804043391762848, 39.42929513812348], [-0.38045260823891347, 39.4293174056924], [-0.380482673469744, 39.4293446503179], [-0.3805383125734109, 39.42937453425541], [-0.38074147380862955, 39.42948930847135], [-0.38081527819129307, 39.4295292050249], [-0.38093880468950053, 39.42960406808915], [-0.3810247486801128, 39.429647066807064], [-0.3811675020033876, 39.42973245272959], [-0.3812729168272948, 39.42979992554392], [-0.38138974262258263, 39.42986359232701], [-0.3815051630337919, 39.429931290329606], [-0.3815929008845587, 39.42999061704962], [-0.38171017858186107, 39.43006146474216], [-0.3818497097531685, 39.43014421874738], [-0.38200762496765306, 39.43025162945717], [-0.3821835580412476, 39.43036276999167], [-0.3823018692824289, 39.43043428000023], [-0.38245648866546467, 39.43052302161386], [-0.3826617975781535, 39.43064545336348], [-0.38279523536352567, 39.43073587972691], [-0.3829471289722957, 39.430843162611396], [-0.3831005217592128, 39.430964406740515], [-0.38319356666218074, 39.43104930176075], [-0.3832729803673384, 39.43110794435417], [-0.38333651536745683, 39.431149166686694], [-0.3834614797933504, 39.43122238383054], [-0.383656606195512, 39.43133700819951], [-0.38383596171781903, 39.43144841248773], [-0.3839989180841611, 39.4315423396277], [-0.38412257901695296, 39.431615968169815], [-0.38419525664813575, 39.4316565939949], [-0.38425860379068477, 39.43168547847996], [-0.38431169219442324, 39.43171004206711], [-0.3844220381542819, 39.43176324489483], [-0.38464278525555873, 39.43186508399132], [-0.38489832534350704, 39.431992407704996], [-0.3851163212980442, 39.432087706652005], [-0.38528439341201676, 39.43216763021525], [-0.3854118952660563, 39.43221752438379], [-0.38545940047541033, 39.43223701600895], [-0.38553764515332095, 39.43226518985703], [-0.3855873577302021, 39.43228250589014], [-0.38565732968609023, 39.43231724122805], [-0.3857309757731678, 39.43235655457531], [-0.3857607906639692, 39.43236883752573], [-0.38581668810276165, 39.43239411224557], [-0.3858762919595828, 39.432423722472265], [-0.38592193466065394, 39.43244611792111], [-0.38593430069474993, 39.43244845904097], [-0.38605261085937376, 39.43248601988655], [-0.38616531378181534, 39.43252285994207], [-0.3863159956642965, 39.43256833750583], [-0.3864470933398372, 39.43261406823017], [-0.38666733286920757, 39.4326845773224], [-0.38686682271495376, 39.43273369196625], [-0.3870179284907358, 39.43276909724394], [-0.3871421505544, 39.43280143862077], [-0.3872523157661196, 39.4328322296972], [-0.3873087924888008, 39.43285073305662], [-0.38737622771206215, 39.432873185332674], [-0.38747004128024753, 39.43291223016254], [-0.38755144088785765, 39.43296109485206], [-0.3876260005760495, 39.433005850877585], [-0.38771370992638754, 39.433066803005545], [-0.38776141397215663, 39.43309655024897], [-0.3877821621081911, 39.433113034263656], [-0.3877966773386915, 39.43312750436421], [-0.3878248267435297, 39.43314737986654], [-0.3878896451088332, 39.43319634917562], [-0.38798874425808866, 39.4332403951689], [-0.38809464527445503, 39.433275585056364], [-0.3884812384281411, 39.43340481863387], [-0.38853619640278353, 39.43341548560964], [-0.3886003243382815, 39.43343288999059], [-0.38865483272524526, 39.433445330564574], [-0.3886781972888568, 39.43345522466125], [-0.38870574768899396, 39.43346688843425], [-0.3890247735634402, 39.43356136864586], [-0.3891818993550241, 39.43360487919437], [-0.38922037510601554, 39.43359619398018], [-0.3892460360990162, 39.433571356332266], [-0.3892749360275427, 39.43351547482003], [-0.38928137001710444, 39.433481935195864], [-0.38929904523644565, 39.43339376606493], [-0.3893503586977322, 39.433397504068424], [-0.38937316468592903, 39.43330415294562], [-0.38950205547108063, 39.432875568195044], [-0.3895617587103193, 39.43259614779459], [-0.38960220268714646, 39.432406878960755], [-0.3896928694097956, 39.432034930085955], [-0.38980678321157064, 39.43156086513992], [-0.38989959219452086, 39.43116072135156], [-0.39000317007554974, 39.430845178259766], [-0.3900109367707682, 39.43063714876903], [-0.39009712116929973, 39.43031791777768], [-0.3902143118399905, 39.42992363628092], [-0.3903151075564109, 39.42943730332506], [-0.39042351478637916, 39.42902990563533], [-0.3905557238560155, 39.428529421952234], [-0.39068274095148137, 39.42803166832378], [-0.39079368496009276, 39.42756737303017], [-0.3908782194228361, 39.427223032524836], [-0.39092299921874196, 39.42704967090429], [-0.3909651010947711, 39.42688020405082], [-0.3909985079419446, 39.42673828935014], [-0.3911577639246907, 39.426761385773716], [-0.3913219446849471, 39.42678418709769], [-0.39146746175552, 39.42681343414341], [-0.3919169354711197, 39.42686835651604], [-0.39204358681308044, 39.426907918148196], [-0.39215236619702465, 39.42692616010528], [-0.3922094129980594, 39.42692586331984], [-0.39233914080773197, 39.42694118770879], [-0.3924598008079327, 39.42695452479464], [-0.3926170052790566, 39.42698469021715], [-0.39281002513995605, 39.427019154152276], [-0.39295078969184055, 39.42704535586543], [-0.3931511026946108, 39.42707427117379], [-0.3932103668957159, 39.42708959095732], [-0.3932581379438109, 39.427108463738186], [-0.39342552386734153, 39.42716280197183], [-0.3937283644037039, 39.42721224016704], [-0.3938381985433043, 39.42720937839008], [-0.39398104916436655, 39.42719178844492], [-0.39402607150072677, 39.42719652742877], [-0.3941284157555928, 39.42720730807518], [-0.3942947260442697, 39.42723411685588], [-0.39435399127239956, 39.427242562310404], [-0.39447496466190995, 39.42725593136128], [-0.39458033505709655, 39.42726375245617], [-0.39474088692726456, 39.42726024279424], [-0.394973223757093, 39.42726369287918], [-0.39518841388503645, 39.42727028976044], [-0.3953784601163168, 39.42726115191973], [-0.3954600958171527, 39.427264368399236], [-0.39554275466645494, 39.42726413031824], [-0.3956170757944284, 39.42727067825044], [-0.39565624774696817, 39.42728060977711], [-0.395699302229092, 39.42729078142343], [-0.3957384884160576, 39.427300019557954], [-0.39576475574344283, 39.427318031191476], [-0.39576747117723793, 39.42734710961797], [-0.395767391109313, 39.42737358491388], [-0.39576565402551095, 39.4274202209901], [-0.3957588548503645, 39.4274820677602], [-0.3957502551581093, 39.42754422553852], [-0.39574771677212145, 39.42761143795385], [-0.3957505871981815, 39.427655690800215], [-0.39576013864329224, 39.42768898537593], [-0.3957650551406378, 39.427726022905006], [-0.3957653541197377, 39.42775573186913], [-0.39576288907624263, 39.427784487032206], [-0.3957583856924319, 39.42782835846937], [-0.39576817984806223, 39.42785201898357], [-0.3959795935407932, 39.427853394754635], [-0.39627257442193004, 39.427855480053644], [-0.39648982757666845, 39.42785476950666], [-0.39662628592784377, 39.42786005983926], [-0.396750245861739, 39.427862520605125], [-0.3969247819479974, 39.427867834492766], [-0.3969722142647678, 39.42787587851137], [-0.39701398974192686, 39.42789077772264], [-0.39703387504729004, 39.427937321425084], [-0.3970493217534897, 39.427985144094066], [-0.3970533418780847, 39.42802225159198], [-0.3970655161513165, 39.42805126251654], [-0.3971122495698953, 39.428143920229786], [-0.3971280554438579, 39.42816712065795], [-0.39715226691679834, 39.42819918484357], [-0.39719135565904984, 39.42825817634497], [-0.397209038929893, 39.42828305841428], [-0.3972597787939291, 39.42836951757853], [-0.3973018995634847, 39.42845316215842], [-0.3973181053509733, 39.42847935345366], [-0.39735171753750304, 39.428490123885005], [-0.39740568891809036, 39.42848978272856], [-0.3974534510439586, 39.42849085205854], [-0.39748320984546104, 39.428484058957245], [-0.3975518559199601, 39.42846491141449], [-0.39760332074061483, 39.428461378960336], [-0.3976387225833454, 39.42846214447497], [-0.3976685841824882, 39.42847100204879], [-0.3977094168579396, 39.42848284391579], [-0.39772022880432534, 39.42847605902904], [-0.3977337344333629, 39.42844246996665], [-0.39776388646078104, 39.42840950579459], [-0.39781332862539426, 39.42840163974177], [-0.397911526910385, 39.42840496401212], [-0.39802353794474254, 39.42842200259018], [-0.3981497065587015, 39.428429132865126], [-0.39827843891129355, 39.428440104112354], [-0.3983530942254719, 39.428453009046784], [-0.39840904816861095, 39.42845776577572], [-0.3984807085393991, 39.428453756996326], [-0.3985526221217457, 39.42846729412777], [-0.3985941015477679, 39.42848298794137], [-0.3986144884423278, 39.428445525795794], [-0.39863795346785597, 39.42839901554311], [-0.3986643311696597, 39.42834696694163], [-0.39870300595177083, 39.42829192329048], [-0.3987355528681935, 39.4282442198705], [-0.3987676305396662, 39.4282091273814], [-0.3988030160337621, 39.42818254121334], [-0.39885644424761885, 39.428157764150434], [-0.3989072347986934, 39.42813081996443], [-0.39897472688525515, 39.42809302439336], [-0.39901835459252816, 39.428070640540085], [-0.3990633593197613, 39.42805339551561], [-0.39910999256827817, 39.42803171850524], [-0.3991660727038673, 39.428008406016644], [-0.39922405341861605, 39.42798302794421], [-0.3992683758273577, 39.42795945234755], [-0.3993158172109011, 39.427925415130105], [-0.3993805932396518, 39.42788754954394], [-0.39942244061839643, 39.42786773828188], [-0.3994834305171096, 39.42785858032142], [-0.39953594525343666, 39.427858349724424], [-0.3995890338526356, 39.42785674457313], [-0.3996552292999418, 39.42782943295741], [-0.39970078142235604, 39.42779746139574], [-0.3997681900802218, 39.42778562708535], [-0.3998414390147143, 39.427771680158045], [-0.3999157699399931, 39.427744568381414], [-0.3999844332757847, 39.427718672115745], [-0.40006891466484584, 39.42769661603088], [-0.40014105105021447, 39.427699463613614], [-0.4002276771335613, 39.427705013551474], [-0.40029119522349177, 39.42770908269198], [-0.4003683884463531, 39.427717142251865], [-0.4006462205277036, 39.42771549939495], [-0.4007661396070122, 39.427721351950545], [-0.40085393522718954, 39.42771856732824], [-0.4009847772933819, 39.427709637066854], [-0.4010805739404787, 39.4277109860589], [-0.4011743450121537, 39.427726514715644], [-0.4012730179471818, 39.427732675534166], [-0.4013454255867595, 39.427728852849796], [-0.4014492141120032, 39.42771176783911], [-0.4015760763756656, 39.427710837924295], [-0.4016163307600249, 39.42772136821421], [-0.40164047041309797, 39.42774572728406], [-0.4016672794419864, 39.427785154817016], [-0.4016926078410218, 39.427907872423454], [-0.40171186045402363, 39.42798059004341], [-0.4017184356836906, 39.42806685293783], [-0.40173057329886436, 39.4281922396195], [-0.4017375763464793, 39.428285746207216], [-0.40175174106812034, 39.42840034951395], [-0.4017585740517452, 39.428465924756686], [-0.40176403176616565, 39.42853568541717], [-0.4017721193150007, 39.42859094650062], [-0.4017844421831272, 39.428681031494826], [-0.40179751107739337, 39.42878917795543], [-0.40179567291151425, 39.42886333403603], [-0.40179831378940534, 39.428978571638325], [-0.4018044327133595, 39.42909305815968], [-0.4018166416816263, 39.42917151914046], [-0.40182677235584185, 39.42923651043181], [-0.40184064452924073, 39.42928490990475], [-0.40184972318578915, 39.4293347607743], [-0.40185887366484485, 39.42939955898512], [-0.40187428834682654, 39.42944990281277], [-0.4018974333782823, 39.4294982302558], [-0.40190734942388556, 39.42952336144354], [-0.4019216775872787, 39.42957105939585], [-0.40193749998345024, 39.429609808886184], [-0.4019641424658507, 39.42966968279171], [-0.40197550160025064, 39.42970693614701], [-0.40199652579244916, 39.42975481977929], [-0.4020246531320034, 39.42984163638936], [-0.40205742411404105, 39.42995542126124], [-0.4020954790867151, 39.43007769346553], [-0.4021249373593081, 39.430152458883484], [-0.40215256174629543, 39.430214678981], [-0.40217562805013596, 39.430259509162475], [-0.4021927498430434, 39.43031050370099], [-0.4022058181146762, 39.430386488378], [-0.4022151587333311, 39.43044805661693], [-0.40222253672420943, 39.43051804934381], [-0.4022248903953086, 39.43064259568409], [-0.40226031397538387, 39.43081662811283], [-0.4022962076561344, 39.430978013555176], [-0.40230952466973796, 39.43110921020096], [-0.4023218449957315, 39.431279735364285], [-0.40234228553544343, 39.431512477139265], [-0.4023464856092501, 39.43167800107879], [-0.40234427354202607, 39.431795679700485], [-0.40235132419339603, 39.43188417834388], [-0.4023842774336592, 39.4320015346972], [-0.4023875058468742, 39.432081912001564], [-0.4023769519725241, 39.432142711323586], [-0.4023527373345154, 39.43223693461781], [-0.4023430595283844, 39.43238607657227], [-0.4023371105242191, 39.43249489702398], [-0.40236226640255873, 39.43259825049816], [-0.4023836000227552, 39.432693482631116], [-0.40241101821155234, 39.432902903142626], [-0.4024484745339784, 39.43312293519497], [-0.40248657839444724, 39.433268334129046], [-0.4025133056844655, 39.433386623365294], [-0.40251813502929484, 39.43347988308815], [-0.40247401302036245, 39.433730596209244], [-0.4024763467394848, 39.433835052250934], [-0.40249417138277466, 39.43396876394049], [-0.40251990360746165, 39.43411227380779], [-0.4025374987648053, 39.43421683663897], [-0.40254383616913714, 39.43426482440238], [-0.4025464847927202, 39.43430265781229], [-0.4025736250756699, 39.434428289478284], [-0.40258198529191985, 39.43450156531433], [-0.40258524563308645, 39.43456326789944], [-0.4025912710594053, 39.43469437360054], [-0.40258354837117305, 39.434839982689745], [-0.40258788000773127, 39.43499322138475], [-0.40259902080578364, 39.43504888393067], [-0.4025991254809018, 39.43509671439783], [-0.40261611197359504, 39.43530676656282], [-0.4026226488295464, 39.4354738200869], [-0.4026196011279043, 39.435558336327645], [-0.40260339870512774, 39.43561449597366], [-0.40256043291716814, 39.43569556170643], [-0.4025139187434392, 39.43580333151775], [-0.4024475012466569, 39.435945854215916], [-0.4024264065086562, 39.436007354875], [-0.40240801130898934, 39.43607874453892], [-0.402384745483687, 39.43617959248346], [-0.4023539796387258, 39.43631062371592], [-0.4023003673053826, 39.43648807159717], [-0.40226520883730954, 39.436621310903156], [-0.4023395598498212, 39.43664484604084], [-0.40251225401475726, 39.43667969574133], [-0.40267464272845865, 39.43673800825605], [-0.4028013226196681, 39.436695217869385], [-0.40301180559026684, 39.436641389965885], [-0.40326531228048346, 39.43658873002194], [-0.4036482573558493, 39.436532204109355], [-0.4040541661022172, 39.43646546920711], [-0.4043480834551357, 39.4364258082796], [-0.40468372423086735, 39.436436410802564], [-0.4048803576988631, 39.43640163934892], [-0.40516782558961956, 39.43636888630024], [-0.40548549378062065, 39.4363394191995], [-0.40595558634813933, 39.43630528994052], [-0.4061476294281079, 39.43627021569159], [-0.406320879308125, 39.43622969483154], [-0.406499722973224, 39.43622064041449], [-0.40690064413750926, 39.43621651306436], [-0.40727212236141186, 39.436214042889056], [-0.407814135315502, 39.43619975323355], [-0.40821494745868586, 39.436198249502844], [-0.4085851105751072, 39.436206493398174], [-0.40887587671721637, 39.436205047476], [-0.4092228501279105, 39.43620234012031], [-0.40955146848743057, 39.436186123759434], [-0.4098858582309754, 39.43615962980418], [-0.41033326251535707, 39.436130400568466], [-0.4105865072683387, 39.43611252904963], [-0.41076484972255867, 39.43610632171028], [-0.41109647719441866, 39.4361252125651], [-0.41144585851867443, 39.436142804166195], [-0.4116465712130253, 39.4361383188428], [-0.41205599800078224, 39.43614041704207], [-0.4124425506030367, 39.43614436493782], [-0.41273940420336774, 39.43614166648], [-0.41300614790549417, 39.43614295474791], [-0.4132068919678011, 39.43613353955952], [-0.4134861194570923, 39.43612115877312], [-0.4137632348186472, 39.43608547837258], [-0.4141377772907334, 39.4360428747265], [-0.4143863923952258, 39.436018846943135], [-0.41461471482485807, 39.435994403317984], [-0.4149742220448283, 39.43596156151861], [-0.41563774545531335, 39.435901316443136], [-0.41611382951243814, 39.4358372254783], [-0.41637724677176463, 39.43580621623294], [-0.41674663863975503, 39.435768354454716], [-0.4169971611144754, 39.435742336443134], [-0.4173499681767255, 39.435711230238816], [-0.4176636827465143, 39.43568008332663], [-0.4181701076279516, 39.43562861272506], [-0.41845992313290165, 39.43560220303141], [-0.41877056926303957, 39.43556383190437], [-0.4190495069030875, 39.435519260479026], [-0.4194426995197775, 39.43546191878408], [-0.41984378572693826, 39.43534241837811], [-0.4201823331103086, 39.43524840217211], [-0.42053575911830376, 39.43514627379326], [-0.4205968290439779, 39.435209664253996], [-0.42067190721751513, 39.435284149174926], [-0.4207786295420027, 39.435358074944226], [-0.4208772761485136, 39.43541850627865], [-0.42096634712998227, 39.43547271613938], [-0.42101726594006594, 39.4354888186566], [-0.4211132243221997, 39.43550592940344], [-0.42120897588652506, 39.4355280174061], [-0.4212686734687488, 39.43558171052266], [-0.4213194037277094, 39.43561867334334], [-0.421421102015056, 39.43566789277478], [-0.42157700829865485, 39.4357423232831], [-0.4216868528935782, 39.43582519125643], [-0.42188073511661195, 39.436028218940116], [-0.4220723269768474, 39.436221871297526], [-0.4221731287164026, 39.43632189842584], [-0.42224262322045064, 39.43642279874913], [-0.42235138814924716, 39.43657405576831], [-0.42239541888223314, 39.43666513018328], [-0.4224602631188715, 39.436762981348444], [-0.42257651774110955, 39.43690554848764], [-0.422720520158662, 39.43712867574155], [-0.42278077634403394, 39.43722274036291], [-0.42283071841651365, 39.437289009037265], [-0.4229124353048387, 39.437401296935754], [-0.42293657982978117, 39.437438102069216], [-0.4230890749077014, 39.43769154292663], [-0.42318506727613564, 39.43781906585066], [-0.4232736264733704, 39.43793077545973], [-0.42345672769923437, 39.43817849797813], [-0.42352622546805385, 39.438279406627615], [-0.42358547480955994, 39.43832803464661], [-0.4236567275709524, 39.438381658356406], [-0.42375142338880956, 39.438461828321586], [-0.42379105907782927, 39.43854550759553], [-0.42390256447696517, 39.4386742836521], [-0.4239991813749299, 39.438785252385806], [-0.42406904080382507, 39.438876700375445], [-0.42413283042015903, 39.43902755364847], [-0.4241760748289396, 39.43919081632084], [-0.4242316420163363, 39.43932299205512], [-0.4242941170732603, 39.43949314020217], [-0.42434288385421626, 39.43957782341575], [-0.424379679901602, 39.43966891695604], [-0.42444721570511335, 39.43979224078073], [-0.42452785677334354, 39.43995372836084], [-0.4245605151952175, 39.440028162797184], [-0.42461631867916794, 39.440212604217685], [-0.4246620670497134, 39.44037921943826], [-0.4246940417122678, 39.440505322751136], [-0.4247454425303759, 39.44076669266153], [-0.42478719438792245, 39.44088444509655], [-0.4248294382983674, 39.440980019495655], [-0.42487814896540654, 39.441066268790394], [-0.4249323011459586, 39.44117953018184], [-0.425019999814525, 39.441299327427984], [-0.42516230382458986, 39.44147596418205], [-0.4252731955318326, 39.441588473295894], [-0.4253741157244248, 39.441675203188], [-0.4254218924061422, 39.44175214339467], [-0.42545960000091404, 39.44183716676622], [-0.4255228222311897, 39.44195310622811], [-0.4256111478409453, 39.44208274563175], [-0.42568357391670636, 39.44221315895079], [-0.425850226224421, 39.44241083838094], [-0.4259520637751481, 39.44254557850704], [-0.4260714463734483, 39.44273236409765], [-0.426195171327093, 39.44290663326054], [-0.4263519775525574, 39.443125345956204], [-0.42645030800970307, 39.443342745735315], [-0.42652034446269116, 39.443481330756924], [-0.4265579246355766, 39.44358627864012], [-0.4266089527258321, 39.44368637115226], [-0.4266768451028395, 39.443786017192004], [-0.42677805189524437, 39.44391936427996], [-0.4268891492051259, 39.444015101861446], [-0.4269216677393317, 39.44404643396692], [-0.42725435935416706, 39.4441934812539], [-0.42731911611890744, 39.4442082297217], [-0.4275289174130519, 39.44432864655014], [-0.4277340188071957, 39.44448395857301], [-0.42799043746520693, 39.444682044926466], [-0.4282956083276797, 39.444933271821604], [-0.4284331748908914, 39.44505151224071], [-0.42859556001614735, 39.44519313911789], [-0.42869947059727065, 39.445293896124916], [-0.428850176481077, 39.44544002098546], [-0.4288759204520247, 39.44551087789353], [-0.42886015742191413, 39.44557864550424], [-0.42879788835293425, 39.44569423061673], [-0.4286861414660853, 39.44589227869861], [-0.42865376177755243, 39.446010894273], [-0.42857436125028137, 39.4461622529743], [-0.42854397367282787, 39.44622199438256], [-0.42852801346274766, 39.446269847907544], [-0.4285331041252308, 39.44630075294678], [-0.42854075099717537, 39.446331660484184], [-0.42853735443876695, 39.4463500716014], [-0.428513400700047, 39.446381487216904], [-0.42844337305680924, 39.44643222558215], [-0.42840940437199926, 39.44649959933849], [-0.42836144943008864, 39.446589284983446], [-0.4283482031531113, 39.44663064004663], [-0.4283482781199607, 39.44667501052889], [-0.42836683108218676, 39.446711411560514], [-0.4284145959629063, 39.44676517039728], [-0.42846638486325234, 39.44684472062874], [-0.42853133892506845, 39.446940688025194], [-0.4285995601831156, 39.4470624931143], [-0.4287704443846446, 39.4472483162521], [-0.4289700958719817, 39.44746835163605], [-0.42904240983912206, 39.447559490621636], [-0.42914685253840446, 39.447684420873955], [-0.4291802622511365, 39.4477304475657], [-0.4292376893653087, 39.44782796879214], [-0.42930491353508576, 39.447938715520834], [-0.42938707918068886, 39.448073197778776], [-0.4294602994020064, 39.448178266289986], [-0.42952414391386223, 39.44828835257178], [-0.4296027491847288, 39.44839424277968], [-0.42970123180343656, 39.448522049709965], [-0.4297679164661792, 39.448620018636205], [-0.42979562940959853, 39.448616559430015], [-0.42984941048523145, 39.44869004507521], [-0.4299070455063252, 39.44877595810398], [-0.4300207414600238, 39.4488569666104], [-0.4301531756906824, 39.44908216230784], [-0.4302606042826284, 39.44926566137533], [-0.4303494419075189, 39.449431136938706], [-0.43055167485289625, 39.449653947412585], [-0.43073188311694544, 39.44976601079105], [-0.4308170993950004, 39.449826515359284], [-0.43092302591364223, 39.4499662514342], [-0.4310795350027449, 39.45017046497884], [-0.43121056069464136, 39.45033457506123], [-0.4313122457339195, 39.45046139738561], [-0.4314387984721046, 39.450622984969115], [-0.43150335306235205, 39.45072464453661], [-0.4315363920360491, 39.450791896341066], [-0.4315940092590144, 39.450874600986616], [-0.43173738775349574, 39.45104255998536], [-0.43179767998004304, 39.45111889128709], [-0.43191677791928296, 39.45123518807073], [-0.43198707525700847, 39.45130931694474], [-0.4318867034128731, 39.45134047721249], [-0.4316921956529366, 39.45138552108159], [-0.4315014257239093, 39.451430665240274], [-0.4312957652152188, 39.45147403857287], [-0.4311711758741756, 39.45151088814437], [-0.430799692999344, 39.45160225256751], [-0.4307019661321431, 39.45164993852577], [-0.4304195697257421, 39.451727568479846], [-0.4301171970756435, 39.45180362093426], [-0.4296664474895948, 39.451951534593455], [-0.42952339664932626, 39.4519939289857], [-0.4294457633049841, 39.4519862661744], [-0.42931661416895506, 39.45198340078362], [-0.4291365939215757, 39.451974761451986], [-0.4290081115902654, 39.45199072103474], [-0.42888421699546325, 39.452008953104006], [-0.4287244533360747, 39.45202172492307], [-0.4285458859828353, 39.45205506221661], [-0.4283361767059263, 39.452088367842144], [-0.42819752622435603, 39.45209927277682], [-0.4280203755329418, 39.45211021747352], [-0.4278363807263245, 39.45213503728016], [-0.42757805036424784, 39.45216172505387], [-0.42725910729435285, 39.45219062009186], [-0.426942745678409, 39.452188860094495], [-0.42667650574753585, 39.45217378572121], [-0.426587724119622, 39.452149523039154], [-0.426438082482239, 39.45216983103093], [-0.4261543785906237, 39.452166071313684], [-0.4260496026522362, 39.45216243573144], [-0.425948264912532, 39.452157578919014], [-0.4258849273022697, 39.452131248600494], [-0.4258929955256138, 39.45205010405502], [-0.42589965808718977, 39.45196106361734], [-0.42589212064350485, 39.45185593412829], [-0.42587138452963513, 39.45181034936256], [-0.4258293572243932, 39.4517569508045], [-0.42579830375851624, 39.45172740757446], [-0.4256823613826038, 39.451705164241204], [-0.4255126036925267, 39.4516804315049], [-0.4254336327084615, 39.4516693580551], [-0.42532402351504744, 39.451640813296464], [-0.4251971893915926, 39.451623084901385], [-0.42508256708078423, 39.45160003238908], [-0.42494694314867926, 39.451559739925415], [-0.4248365137966736, 39.45151247408896], [-0.42471509357695747, 39.4514780815638], [-0.4245914190262885, 39.45143990928177], [-0.42446238813219167, 39.45141653689369], [-0.42430950038412085, 39.451426265859105], [-0.4241912382686223, 39.45143670793237], [-0.424035699828047, 39.45147987283192], [-0.42383887821663, 39.451533320548], [-0.4236973830173944, 39.45156024692791], [-0.42358139857467225, 39.451574459532246], [-0.4234562009946461, 39.45157727874879], [-0.4233528519901022, 39.45157499668012], [-0.42323412427933954, 39.451559301685], [-0.4231030223057409, 39.451526557676665], [-0.4229900599171883, 39.45152405313863], [-0.4227672948058975, 39.451498600564804], [-0.4225589126556577, 39.45148425916032], [-0.4225186617915931, 39.451481366592866], [-0.42246005336917963, 39.45150328277357], [-0.4223323265869883, 39.451509774367416], [-0.4221950427909113, 39.45151419802797], [-0.4221418917873434, 39.45152047854772], [-0.42206285858237175, 39.4515303200586], [-0.42197032564495807, 39.45154448363972], [-0.4219168307764694, 39.45156008062293], [-0.4218887771669482, 39.451611772942464], [-0.42188161256148593, 39.45164406401824], [-0.4218673288989131, 39.45167435036036], [-0.421831781006007, 39.4516976786251], [-0.42177255446790796, 39.451713977217494], [-0.4215935656528977, 39.45172120471252], [-0.4213830897562458, 39.45172642708145], [-0.4210843236802613, 39.45173191528054], [-0.42095842711721493, 39.451741382860945], [-0.4207638550213229, 39.451738858016505], [-0.42056786513741556, 39.4517498687563], [-0.42039272718400883, 39.45176892743867], [-0.4203044064207715, 39.45180117390503], [-0.42020182785677446, 39.45185291436643], [-0.42008313044728474, 39.451922305697714], [-0.4199684099567775, 39.45198878522712], [-0.4198197540519093, 39.45203050263759], [-0.4197320986185032, 39.45209757374156], [-0.4196564968737954, 39.45215291255088], [-0.4195569517822985, 39.45218133272375], [-0.4195518354149979, 39.45226164139693], [-0.4195641007940114, 39.452348939480366], [-0.41961769031568097, 39.45247017255262], [-0.4196330352589221, 39.45257854770291], [-0.4195200313233598, 39.45259159718119], [-0.41942814769547676, 39.45260669203388], [-0.4192372329172255, 39.45263658768013], [-0.4191218790775963, 39.452656935834966], [-0.4189882813692582, 39.45270368916259], [-0.41883007568974184, 39.45276006686711], [-0.4187210715686144, 39.452806690386964], [-0.41858128054391547, 39.452867566744615], [-0.41832323124074866, 39.45293679801897], [-0.4180749102473313, 39.4529905603696], [-0.4179573315260149, 39.45302297486944], [-0.4178564103979284, 39.45303701185071], [-0.417584275275724, 39.45311127980078], [-0.41744476324435137, 39.4531504225526], [-0.41740544501966304, 39.45314114361694], [-0.41735693391223644, 39.45313629095799], [-0.41708514845808187, 39.45324161908711], [-0.41696059017303533, 39.453368794958486], [-0.41691417453511315, 39.453423231039565], [-0.4167571528377883, 39.45350186609076], [-0.41664186452612656, 39.45355934778787], [-0.41655670049415777, 39.45359379661821], [-0.4164713952200804, 39.45363206200764], [-0.4164091977365624, 39.453645976507275], [-0.4163438318208606, 39.45367894637808], [-0.41621508696422527, 39.45373766001817], [-0.416051095314009, 39.45376843716813], [-0.41598021699597765, 39.45381659035252], [-0.4158628641957888, 39.45385414278086], [-0.41579778023586084, 39.453879479083724], [-0.41578024143294817, 39.45393484502899], [-0.4158099243939281, 39.45399914367909], [-0.4158486296761155, 39.45408499394898], [-0.4158845382336755, 39.45418142150243], [-0.4159125556483524, 39.45429106078192], [-0.4159340958308973, 39.45437569155936], [-0.41598373868415295, 39.4544231639273], [-0.41600453324240416, 39.45445947254504], [-0.4160113422012219, 39.45449028111524], [-0.4160133931965908, 39.45453284858751], [-0.41598802164090837, 39.45458602243027], [-0.4160159371113398, 39.454672038155095], [-0.41604806699347363, 39.45477921030966], [-0.41606776096993286, 39.45484653000865], [-0.4160640396017654, 39.454880843145006], [-0.4160338413181834, 39.45489736970492], [-0.41590786403807084, 39.454904136285826], [-0.41572789725172, 39.45490395462646], [-0.415629433622374, 39.454899855366136], [-0.415562935213061, 39.454898376699106], [-0.4155709448590109, 39.45494823872489], [-0.4156092838280959, 39.45511152116924], [-0.41561441377799785, 39.455187390828286], [-0.41565137346448916, 39.455256355752184], [-0.4156706270449144, 39.45534593564576], [-0.41566351275076024, 39.45544587500925], [-0.4156245640799392, 39.45556540348994], [-0.4156101063245853, 39.45562322525123], [-0.41557040305832643, 39.455696872775356], [-0.4155410408930159, 39.45575737206604], [-0.41553077117471676, 39.45583548482777], [-0.41551779257017, 39.45591614091332], [-0.4154833063931489, 39.45598226489311], [-0.4154531669312918, 39.456063764563154], [-0.41543520606342205, 39.4561493548086], [-0.41540479229280747, 39.45624297429991], [-0.41536954559159905, 39.456303576908915], [-0.4153454801390586, 39.45640464374883], [-0.4153165677882269, 39.45653744914131], [-0.4152756588474467, 39.45669169910101], [-0.41523646357706345, 39.456799951838946], [-0.4152309784206193, 39.45688850413499], [-0.4151831773489958, 39.4570773029286], [-0.4151642060043391, 39.457107907542635], [-0.41513983184643416, 39.4571667428973], [-0.41513863503571813, 39.4571893465537], [-0.4151383713770084, 39.45722567345033], [-0.4151331060857073, 39.45738960781324], [-0.4151328417470502, 39.457428150874925], [-0.41513641296837067, 39.45746668014359], [-0.41513595036021383, 39.45753475886883], [-0.41515339450018574, 39.45761638879215], [-0.41511659556402886, 39.457730523500096], [-0.41509035118449344, 39.45785464053001], [-0.4150786874481338, 39.45795846054639], [-0.41507576691596504, 39.45804647527265], [-0.41507285675141403, 39.458075806476636], [-0.41503786875825954, 39.45814351369679], [-0.414999990234434, 39.45822023755103], [-0.4149941593136987, 39.45826536840123], [-0.4149941492127368, 39.45836015044955], [-0.4149416956687331, 39.4583781996845], [-0.4149067251723729, 39.45839173694226], [-0.4148600927048291, 39.458423320806666], [-0.41478432519472114, 39.45846620234053], [-0.41476975317752923, 39.458488769730394], [-0.41475518455262933, 39.458531652183915], [-0.4147432708325129, 39.45860107975325], [-0.41471997005478944, 39.45868457811928], [-0.4146266872768254, 39.458932805362686], [-0.4145917199504604, 39.45897795469536], [-0.4145625619161466, 39.459025341322494], [-0.4145275802293101, 39.45913366046779], [-0.41447220637117377, 39.459309678000345], [-0.414431265158876, 39.45941509880965], [-0.4144021124668418, 39.45948053927117], [-0.41438753301403686, 39.45954598862914], [-0.4143525635090557, 39.459593381039404]]], "type": "Polygon"}
"""

def update_district_geometry(graph, district_slug, new_polygon_json):
    """
    Actualiza la geometría de un distrito en el grafo RDF.
    Elimina la geometría existente y añade la nueva.
    """
    if not new_polygon_json:
        print(f"⚠️ Saltando actualización para '{district_slug}': JSON no definido.")
        return

    place_uri = URIRef(domain + 'district/' + district_slug)
    shape_uri = URIRef(place_uri + "/geoshape")
    
    # 1. Eliminar tripletas de geometría existentes para este shape
    # (Eliminamos todo lo que tenga shape_uri como sujeto)
    graph.remove((shape_uri, None, None))
    
    # 2. Asegurar que el Place apunta al Shape
    graph.add((place_uri, schema.geo, shape_uri))
    
    # 3. Añadir los nuevos datos
    graph.add((shape_uri, RDF.type, schema.GeoShape))
    graph.add((shape_uri, schema.polygon, Literal(new_polygon_json.strip())))
    
    print(f"✅ Geometría actualizada manualmente para: {district_slug}")

# Aplicar correcciones
update_district_geometry(g, 'poblats-maritims', POLY_MARITIMS)
update_district_geometry(g, 'poblats-del-sud', POLY_SUD)

# --- SERIALIZACIÓN FINAL ---
# Guardamos el grafo AHORA, después de las correcciones
output_dir = "../rdf"
if not os.path.exists("../rdf"):
    output_dir = "rdf"
output_file_name = "valencia_districts_places_enriched.ttl"
output_file = os.path.join(output_dir, output_file_name)

os.makedirs(output_dir, exist_ok=True) 
g.serialize(destination=output_file)

print(f"\n💾 Grafo RDF FINAL guardado en: {output_file}")
print(f"Total de triples (final): {len(g)}")

✅ Geometría actualizada manualmente para: poblats-maritims
✅ Geometría actualizada manualmente para: poblats-del-sud

💾 Grafo RDF FINAL guardado en: ../rdf\valencia_districts_places_enriched.ttl
Total de triples (final): 266


## 7. Visualización (SPARQL)

Para verificar que los enlaces con Wikidata son correctos, puedes ejecutar la siguiente consulta **SPARQL** en el [Servicio de Consultas de Wikidata](https://query.wikidata.org/).

Esta consulta:
1.  Selecciona los distritos que hemos enlazado.
2.  Obtiene sus coordenadas desde Wikidata.
3.  Los muestra en un mapa interactivo.

Copia y pega el siguiente código en el editor de Wikidata:

In [17]:
#defaultView:Map
SELECT ?r ?rLabel ?location
WHERE {
  # Lista de distritos de Valencia (QIDs)
  VALUES ?r { 
    wd:Q3392835 wd:Q777986 wd:Q536707 wd:Q3392782 wd:Q3393032 
    wd:Q3392733 wd:Q4452090 wd:Q3393056 wd:Q3393002 wd:Q3392791 
    wd:Q3392792 wd:Q3393007 wd:Q55226 wd:Q3393115 wd:Q3393120 
    wd:Q3392798 wd:Q3392780 wd:Q3393088 wd:Q3392759 
  }
  
  ?r wdt:P625 ?location. 
  
  SERVICE wikibase:label { 
    bd:serviceParam wikibase:language "[AUTO_LANGUAGE],es". 
    ?r rdfs:label ?rLabel. 
  }
}

SyntaxError: invalid syntax (1229766311.py, line 2)